<center>

# ~~~ Symmetry Adapted Spherical Harmonics ~~~

<div style="max-width:600px; word-wrap:break-word;">

To obtain symmetry adapted linear combinations of spherical harmonics, it proves useful to start with real spherical harmonics. In which case if the irreducible representation that is used to generate symmetry-adapted combination is real, then the results are guaranteed to be real as well.

Here's the development code used to generate `symmetry_bases.pkl`. Here also is the code used to generate the LaTeX output.

The generation of this was summarized in the script `gen_bases.py`.

</center>

<center>

$Y_{l,0}^{(c)} = Y_{l,0}$

$Y_{l,m}^{(c)} = \frac{1}{\sqrt{2}} \left( Y_{l,m} + Y_{l,-m} \right)$ for $m > 0$

$Y_{l,m}^{(s)} = -\frac{i}{\sqrt{2}} \left( Y_{l,m} - Y_{l,-m} \right)$ for $m < 0$

$Y_{l,m} = \frac{1}{\sqrt{2}} \left( Y_{l,m}^{(c)} + i Y_{l,m}^{(s)} \right)$ for $m > 0$

$Y_{l,-m} = \frac{1}{\sqrt{2}} \left( Y_{l,m}^{(c)} - i Y_{l,m}^{(s)} \right)$ for $m > 0$

</center>

In [1]:
%load_ext autoreload
# provide cell timings
%load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
# from sympy import poly, lambdify
from IPython.display import display, Math, Latex
%config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from qdef import *
from itertools import product
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()
import warnings

Reloading /Users/juan/Zia Lab/Codebase/qdef/data/CPGs.pkl ...


# Making a Pretty Print of Final Results

In [579]:
sym_bases = pickle.load(open('./data/symmetry_bases.pkl','rb'))

In [724]:
# For each group
# For each irrep
# For each l
numeric = False
sig_figs = 4
final_out_file = '/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/surface harmonics/surface-harmonies.tex'
final_out_file_data = '/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/surface harmonics/surface-harmonies_data.pkl'

out = []
small_rule = '\\noindent\\hfil\\rule{0.1\\textwidth}{.4pt}\\hfil\n'
def centered(string):
    return '\n\\begin{center}\n%s\n\\end{center}\n\n' % string
muticoldict = OrderedDict()
target_groups = CPGs.all_group_labels
raw_dict = {}
# target_groups = ['O']

for group_label in target_groups:
    raw_dict[group_label] = {}
    out.append(('\n\\newpage\n\n\\section{Group $%s$}' % group_label))
    cos_sin = sym_bases[group_label][0]
    for irrep_label, l_dict in cos_sin.items():
        raw_dict[group_label][irrep_label] = {}
        max_len = 0
        line = '\\subsection{$%s$}\n' % str(irrep_label)
        out.append(line)
        # out.append(centered('$%s$' % str(irrep_label)))
        for l, qet_groups in l_dict.items():
            raw_dict[group_label][irrep_label][l] = []
            if len(qet_groups) !=0:
                # out.append(centered('$l=%d$' % l ))
                out.append('\\subsubsection{$%s: l=%d$}' % (str(irrep_label),l))
            for qet_group in qet_groups:
                raw_out = [qet.dict for qet in qet_group]
                if len(raw_out) != 0:
                    raw_dict[group_label][irrep_label][l].append(raw_out)
                qet_group_len = len(qet_group)
                if qet_group_len == 0:
                    out.pop()
                    continue
                out.append('\\begin{subequations}\n\\begin{dmath}')
                for qetidx, qet in enumerate(qet_group):
                    pretty_dict = qet.dict
                    if numeric:
                        pretty_out = sp.latex(sum([(1 if v == 1 else sp.N(v,sig_figs))*sp.Symbol('Y_{%d,%d}^{(%s)}' % k) for k,v in pretty_dict.items()]))
                    else:
                        pretty_out = sp.latex(sum([v*sp.Symbol('Y_{%d,%d}^{(%s)}' % k) for k,v in pretty_dict.items()]))
                    max_len = max(max_len, len(pretty_out))
                    if qet_group_len > 1:
                        if qetidx != qet_group_len - 1:
                            out.append(pretty_out + r' \\ ')
                        else:
                            out.append(pretty_out)
                    else:
                        out.append(pretty_out)
                out.append('\\end{dmath}\n\\end{subequations}\n')
        muticoldict[(group_label, irrep_label)] = max_len
out = []
max_line_threshold = 80

for group_label in target_groups:
    out.append(('\n\\newpage\n\n\\section{Group $%s$}' % group_label))
    cos_sin = sym_bases[group_label][0]
    in_three_mode =  False
    exit_three_mode = False
    for irrep_label, l_dict in cos_sin.items():
        max_len = muticoldict[(group_label, irrep_label)]
        if max_len < max_line_threshold and not(in_three_mode):
            in_three_mode = True
            out.append('\\begin{multicols}{3}\n')
        elif max_len < max_line_threshold and (in_three_mode):
            pass
        elif max_len > max_line_threshold and not(in_three_mode):
            pass
        elif max_len > max_line_threshold and (in_three_mode):
            out.append('\\end{multicols}\n')
            in_three_mode = False
        line = '\\subsection{$%s$}\n' % str(irrep_label)
        out.append(line)
        # out.append(centered('$%s$' % str(irrep_label)))
        for l, qet_groups in l_dict.items():
            if len(qet_groups) !=0:
                # out.append(centered('$l=%d$' % l ))
                out.append('\\subsubsection{$%s: l=%d$}' % (str(irrep_label),l))
            else:
                continue
            for qet_group in qet_groups:
                qet_group_len = len(qet_group)
                if qet_group_len == 0:
                    out.pop()
                    continue

                # out.append('\\begin{subequations}\n\\begin{dmath}')
                out.append(small_rule)
                out.append('\\begin{dgroup}')
                for qetidx, qet in enumerate(qet_group):
                    out.append('\\begin{dmath}')
                    pretty_dict = [(k,v) for k,v in qet.dict.items()]
                    pretty_dict = list(sorted(pretty_dict, key= lambda x: x[0][1]))
                    pretty_dict = OrderedDict(pretty_dict)
                    if numeric:
                        pretty_out = sp.latex(sum([(v if v in [1,-1,-sp.I, sp.I] else sp.N(v,sig_figs))*sp.Symbol('Y_{%d,%d}^{(%s)}' % k) for k,v in pretty_dict.items()]))
                    else:
                        pretty_out = sp.latex(sum([v*sp.Symbol('Y_{%d,%d}^{(%s)}' % k) for k,v in pretty_dict.items()]))
                    if qet_group_len > 1:
                        if qetidx != qet_group_len - 1:
                            out.append(pretty_out + r' \\ ')
                        else:
                            out.append(pretty_out)
                    else:
                        out.append(pretty_out)
                    out.append('\\end{dmath}')
                # out.append('\\end{dmath}\n\\end{subequations}\n')
                out.append('\\end{dgroup}\n\n')
                out.append(r'\vspace*{-0.1cm}')
            # out.append(r'\vspace*{-0.3cm}')
            out.append(small_rule)
            
        # if max_len > max_line_threshold and not exit_three_mode:
        #     out.append('\n\\end{multicols}{3}\n')
        #     exit_three_mode = True
    else:
        if in_three_mode:
            out.append('\\end{multicols}\n')
open(final_out_file,'w').write('\n'.join(out))
pickle.dump(raw_dict, open(final_out_file_data,'wb'))

# Analysis of End Results

In [ ]:
sym_bases_new = pickle.load(open('./data/symmetry_bases_sc_and_standard_12.pkl','rb'))

In [576]:
bummer = []
okay = []
for group_label, sym_basis in sym_bases_new.items():
    for irrep_label, l_dict in sym_basis[0].items():
        for l, qet_groups in l_dict.items():
            for qet_group in qet_groups:
                all_coeffs = sum([list(qet.dict.values()) for qet in qet_group],[])
                reality_check = [sp.im(coeff) == 0 for coeff in all_coeffs]
                if not all(reality_check):
                    bummer.append((group_label,irrep_label,l,qet_group_idx,qet_group))
                else:
                    okay.append((group_label,irrep_label,l,qet_group_idx,qet_group))

print("%d Failed out of %d (%.1f%%)" % (len(bummer),len(okay)+len(bummer), 100*len(bummer)/(len(okay)+len(bummer))))
failed_group_counts = {}
for bumm in bummer:
    group_label = bumm[0]
    if group_label not in failed_group_counts:
        failed_group_counts[group_label] = 0
    failed_group_counts[group_label] += 1
success_group_counts = {}
for ok in okay:
    group_label = ok[0]
    if group_label not in success_group_counts:
        success_group_counts[group_label] = 0
    success_group_counts[group_label] += 1
all_good = []
some_bad = []
for group_label in success_group_counts:
    successful = success_group_counts[group_label]
    total = success_group_counts[group_label]
    if group_label in failed_group_counts:
        total += failed_group_counts[group_label]
        failed = failed_group_counts[group_label]
        some_bad.append(group_label)
    else:
        all_good.append(group_label)
        failed = 0
    print("%s: %d failed out of %d" % (group_label, failed, total))
all_good = set(all_good)
some_bad = set(some_bad)

868 Failed out of 4386 (19.8%)
C_{1}: 0 failed out of 169
C_{i}: 0 failed out of 169
C_{2}: 0 failed out of 169
C_{s}: 0 failed out of 169
C_{2h}: 0 failed out of 169
D_{2}: 0 failed out of 169
C_{2v}: 0 failed out of 169
D_{2h}: 0 failed out of 169
C_{4}: 84 failed out of 169
S_{4}: 84 failed out of 169
C_{4h}: 84 failed out of 169
D_{4}: 0 failed out of 127
C_{4v}: 0 failed out of 127
D_{2d}: 0 failed out of 127
D_{4h}: 0 failed out of 127
C_{3}: 112 failed out of 169
S_{6}: 112 failed out of 169
D_{3}: 0 failed out of 113
C_{3v}: 0 failed out of 113
D_{3d}: 0 failed out of 113
C_{6}: 112 failed out of 169
C_{3h}: 112 failed out of 169
C_{6h}: 112 failed out of 169
D_{6}: 0 failed out of 113
C_{6v}: 0 failed out of 113
D_{3h}: 0 failed out of 113
D_{6h}: 0 failed out of 113
T: 28 failed out of 85
T_{h}: 28 failed out of 85
O: 0 failed out of 71
T_{d}: 0 failed out of 71
O_{h}: 0 failed out of 71


In [560]:
cyclic_groups = set('C_{i} C_{3h} C_{2} C_{1h} C_{4} S_{4} C_{3} C_{6}'.split(' '))
dihedral_groups = set('D_{2} C_{2v} D_{3} D_{2d} C_{3v} D_{6} D_{4} C_{6v} C_{4v} D_{3h}'.split(' '))
print("these are bad and not cyclic")
print([c for c in some_bad if c not in cyclic_groups])
print("these are cyclic and not bad")
print([c for c in cyclic_groups if c not in some_bad])
print("these both cyclic and bad")
print([c for c in cyclic_groups if c in some_bad])
print("these are bad and not dihedral")
print([c for c in some_bad if c not in dihedral_groups])
print("these are dihedral and not bad")
print([c for c in dihedral_groups if c not in some_bad])
print("these both dihedral and bad")
print([c for c in dihedral_groups if c in some_bad])

these are bad and not cyclic
['T', 'S_{6}', 'T_{h}', 'C_{4h}', 'C_{6h}']
these are cyclic and not bad
['C_{2}', 'C_{1h}', 'C_{i}']
these both cyclic and bad
['C_{6}', 'C_{4}', 'S_{4}', 'C_{3h}', 'C_{3}']
these are bad and not dihedral
['C_{6}', 'T', 'S_{6}', 'S_{4}', 'C_{4}', 'T_{h}', 'C_{3h}', 'C_{3}', 'C_{4h}', 'C_{6h}']
these are dihedral and not bad
['D_{2d}', 'C_{6v}', 'D_{3h}', 'D_{6}', 'D_{2}', 'C_{3v}', 'D_{4}', 'D_{3}', 'C_{4v}', 'C_{2v}']
these both dihedral and bad
[]


In [561]:
print([c for c in some_bad if c not in dihedral_groups and c not in cyclic_groups])

['T', 'S_{6}', 'T_{h}', 'C_{4h}', 'C_{6h}']


In [130]:
for c in cyclic_groups:
    if c in some_bad:
        print(c)

C_{6}
C_{4}
S_{4}
C_{3h}
C_{3}


In [112]:
{ir: [v for k,v in irv.items() if k==1] for ir, irv in sym_bases_new['C_{4}'][0].items()}

{A: [[[Qet({(1, 0, 'c'): 1})]]],
 E^1: [[[Qet({(1, 1, 'c'): sqrt(2)/2, (1, 1, 's'): sqrt(2)*I/2})]]],
 E^2: [[[Qet({(1, 1, 'c'): sqrt(2)/2, (1, 1, 's'): -sqrt(2)*I/2})]]],
 B: [[]]}

[('O_{h}',
  E_{g},
  4,
  0,
  [Qet({(4, 0, 'c'): sqrt(30)/12, (4, 2, 'c'): sqrt(2)*I/2, (4, 4, 'c'): -sqrt(42)/12}),
   Qet({(4, 0, 'c'): -sqrt(30)/24 + sqrt(10)*I/8, (4, 2, 'c'): sqrt(6)*(3 + sqrt(3)*I)/12, (4, 4, 'c'): sqrt(42)/24 - sqrt(14)*I/8})])]

In [312]:
vdim = len(v1)
complement = [sp.Symbol('c_%d' % i) for i in range(vdim)]
vmat = sp.Matrix([v1,v2,complement])
detmat = sp.simplify(vmat.det())

In [390]:
A = sp.Matrix([v1,v2])
Ap = A.pinv()
ApA = sp.simplify(Ap*A)

In [448]:
eigensys = ApA.eigenvects()
nicevects = [[(ve.T) for ve in v[2]] for v in eigensys if v[0]==1][0]
nicevects = [v/v.norm() for v in nicevects]
realities = [all([sp.im(c) == 0 for c in nicevect]) for nicevect in nicevects]
irealities = [all([sp.re(c) == 0 for c in nicevect]) for nicevect in nicevects]
bettervects = []
for nicevect, reality, ireality in zip(nicevects, realities, irealities):
    if reality:
        bettervects.append(list(nicevect))
        continue
    if ireality:
        bettervects.append([list(sp.I*v) for v in nicevect])
        continue


In [469]:
# def realize(vs):
#     '''
#     vs are lists of lists where each inner list has the numeric
#     components of each of the provided vectors.
#     '''
#     A = sp.Matrix(vs)
#     Ap = A.pinv()
#     ApA = sp.simplify(Ap*A)
#     eigensys = ApA.eigenvects()
#     nicevects = [[(ve.T) for ve in v[2]] for v in eigensys if v[0]==1][0]
#     nicevects = [list(v/v.norm()) for v in nicevects]
#     realities = [all([sp.im(c) == 0 for c in nicevect]) for nicevect in nicevects]
#     irealities = [all([sp.re(c) == 0 for c in nicevect]) for nicevect in nicevects]
#     bettervects = []
#     for nicevect, reality, ireality in zip(nicevects, realities, irealities):
#         if reality:
#             bettervects.append(list(nicevect))
#             continue
#         if ireality:
#             bettervects.append([list(sp.I*v) for v in nicevect])
#             continue
#     if len(vs) == len(bettervects):
#         return bettervects, "real"
#     else:
#         return vs, "non real"

In [494]:
# def qetrealsimpler(qets):
#     '''
#     Given  a  list of qets, return a set that spans the same subspace, but
#     with real components.

#     Parameters
#     ----------
#     qets  (list) with qdefcore.Qet values

#     Returns
#     -------
#     (2-tuple)  :  (list), (str) If the string is "real" then a solution to
#     make  for  a  real combination was succesful and the returned list has
#     the  resultant  qets,  if  the  string  is  "non-real" then a solution
#     couldn't be found and the qets returned are the original ones.
#     '''
#     # determine the common basis
#     common_basis = list(set(sum([list(q.dict.keys()) for q in qets],[])))
#     # find the coefficients in that basis
#     vs = [q.vec_in_basis(common_basis) for q in qets]
#     A = sp.Matrix(vs)
#     Ap = A.pinv()
#     ApA = sp.simplify(Ap*A)
#     eigensys = ApA.eigenvects()
#     eigensys = (ApA - sp.eye(ApA.rows)).nullspace()
#     nicevects = [[(ve.T) for ve in v[2]] for v in eigensys if v[0]==1][0]
#     nicevects = [list(v/v.norm()) for v in nicevects]
#     realities = [all([sp.im(c) == 0 for c in nicevect]) for nicevect in nicevects]
#     irealities = [all([sp.re(c) == 0 for c in nicevect]) for nicevect in nicevects]
#     bettervects = []
#     for nicevect, reality, ireality in zip(nicevects, realities, irealities):
#         if reality:
#             bettervects.append(list(nicevect))
#             continue
#         if ireality:
#             bettervects.append([list(sp.I*v) for v in nicevect])
#             continue
#     if len(vs) == len(bettervects):
#         betterqets = [Qet({cb:v for cb, v in zip(common_basis, bettervect)}) for bettervect in bettervects]
#         betterqets = sorted(betterqets, key= lambda x: len(x.dict))
#         return betterqets, "real"
#     else:
#         return qets, "non real"

In [501]:
# qets = [Qet({(5, 1, 'c'): 7*sp.sqrt(2)/16, (5, 2, 'c'): sp.sqrt(14)*I/7, (5, 3, 'c'): 5*sp.sqrt(21)/112, (5, 4, 'c'): -sp.sqrt(42)*I/14, (5, 5, 'c'): -3*sp.sqrt(105)/112})]
# # determine the common basis
# common_basis = list(set(sum([list(q.dict.keys()) for q in qets],[])))
# # find the coefficients in that basis
# vs = [q.vec_in_basis(common_basis) for q in qets]
# A = sp.Matrix(vs)

In [ ]:
def irrep_check(group, a_basis_list, irrep, l):
    '''
    This  function  checks  if  a  list  of qets (which are interpreted as
    superpositions  of Ylms for fixed l), are a basis for the given irrep.
    This  is done by calculating the matrices that represent the effect of
    applying  all  of  the  group  operations, and then checking to see if
    those matrices are unitary, satisfy the multiplication table, and have
    the corresponding character for the given irrep.

    This function could easily be modified to return the matrices that the
    given basis induces.

    Parameters
    ----------

    group    (qdef.CrystalGroup)

    a_basis_list   (list[list[qdef.Qet]]): interpreted  as  superpositions 
        of standard spherical harmonics. Each list must have as many qets
        as the dimension of the corresponding irrep. Several groups may be
        given to account for possible degeneracy of the bases.

    irrep   (sp.Symbol): symbol for one of the group irreps.

    l      (int): the  value  of  l  from which the given qets are seen to
        originate from.
    
    Returns
    -------
    all_checks (list): a list of nested lists, of which there are as many
    as groups of qets were given. The enclosed lists have three Bool, the
    first one of which is the check for the multiplication table, the 2nd
    of which is the of characters of the induced matrices, and the  third
    being the check against the matrices being unitary.
    '''
    all_checks = []
    for a_basis in a_basis_list:
        lbasis = [(l,m) for m in range(-l,l+1)]
        basis_matrix = sp.Matrix([b.vec_in_basis(lbasis) for b in a_basis]).T
        basis_matrix = Dagger(basis_matrix)
        induced_matrices = {}
        char_checks = {}
        unitarity_checks = {}
        for group_op, op_params in group.euler_angles.items():
            D_matrix = (op_params[3]**l)*sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)]).T
            R_rows = []
            for basis_element in a_basis:
                vector = sp.Matrix(basis_element.vec_in_basis(lbasis))
                t_vector = D_matrix*vector
                row = (basis_matrix*t_vector).T
                R_rows.append(row)
            induced_matrices[group_op] = sp.Matrix(R_rows).T
            induced_matrices[group_op] = sp.re(induced_matrices[group_op]) + sp.I * sp.im(induced_matrices[group_op])
            diff = sp.N(sp.simplify(induced_matrices[group_op]).trace() - sp.simplify(group.irrep_matrices[irrep][group_op]).trace(), chop=True)
            char_check = (diff == 0)
            if not char_check:
                warnings.warn("Character mismatch.")
            char_checks[group_op] = char_check
            ru = sp.simplify(induced_matrices[group_op] * Dagger(induced_matrices[group_op]))
            ru = sp.re(ru) + sp.I*sp.im(ru)
            unitarity_checks[group_op] =  (ru == sp.eye(group.irrep_dims[irrep]) )
        # check to see if the product table is satisfied
        ptable = group.multiplication_table_dict
        checks = {}
        for group_op_0 in group.group_operations:
            for group_op_1 in group.group_operations:
                group_op_01 = ptable[(group_op_0,group_op_1)]
                target = induced_matrices[group_op_01]
                directp = sp.simplify(induced_matrices[group_op_0]*induced_matrices[group_op_1])
                directp = sp.re(directp) + sp.I * sp.im(directp)
                target = sp.re(target) + sp.I * sp.im(target)
                pcheck = sp.N(directp - target, chop=True) == sp.zeros(target.rows)
                if not pcheck:
                    warnings.warn("Multiplication table mismatch.")
                checks[(group_op_0,group_op_1)] = pcheck
        all_checks.append((all(checks.values()), all(char_checks.values()), all(unitarity_checks.values())))
    return all_checks

In [547]:
group = CPGs.get_group_by_label('T_{d}')

[(False, False, False)]

In [554]:
odd_bases = pickle.load(open('./data/symmetry_bases_sc_and_standard_6.pkl','rb'))

In [555]:
odd_bases['T_{d}'][0][sp.Symbol('E')]

{0: [],
 1: [],
 2: [[Qet({(2, 2, 'c'): 1}), Qet({(2, 0, 'c'): 1})]],
 3: [],
 4: [[Qet({(4, 2, 'c'): 1}),
   Qet({(4, 0, 'c'): -sqrt(15)/6, (4, 4, 'c'): sqrt(21)/6})]]}

In [556]:
irrep_check(group, odd_bases['T_{d}'][1][sp.Symbol('E')][4], sp.Symbol('E'),4)

[(True, True, True)]

In [502]:
Ap = A.pinv()

In [503]:
ApA = sp.simplify(Ap*A)

In [522]:
for rowidx in range(ApA.rows):
    print(str(list(ApA.row(rowidx))).replace('sqrt','Sqrt').replace('(','[').replace(')',']'))

[3/14, -9*Sqrt[10]*I/224, Sqrt[21]*I/16, -Sqrt[3]/7, 15*Sqrt[2]*I/224]
[9*Sqrt[10]*I/224, 135/1792, -3*Sqrt[210]/256, -3*Sqrt[30]*I/112, -45*Sqrt[5]/1792]
[-Sqrt[21]*I/16, -3*Sqrt[210]/256, 49/128, Sqrt[7]*I/8, 5*Sqrt[42]/256]
[-Sqrt[3]/7, 3*Sqrt[30]*I/112, -Sqrt[7]*I/8, 2/7, -5*Sqrt[6]*I/112]
[-15*Sqrt[2]*I/224, -45*Sqrt[5]/1792, 5*Sqrt[42]/256, 5*Sqrt[6]*I/112, 75/1792]


In [539]:


ApAm = ApA - sp.eye(ApA.rows)
eigensys = ApAm.nullspace()
# eigensys = ApA.eigenvects()
nicevects = [(ve.T) for ve in eigensys]
nicevects = [list(v/v.norm()) for v in nicevects]
realities = [all([sp.im(c) == 0 for c in nicevect]) for nicevect in nicevects]
irealities = [all([sp.re(c) == 0 for c in nicevect]) for nicevect in nicevects]
bettervects = []
for nicevect, reality, ireality in zip(nicevects, realities, irealities):
    if reality:
        bettervects.append(list(nicevect))
        continue
    if ireality:
        bettervects.append([list(sp.I*v) for v in nicevect])
        continue
if len(vs) == len(bettervects):
    betterqets = [Qet({cb:v for cb, v in zip(common_basis, bettervect)}) for bettervect in bettervects]
    betterqets = sorted(betterqets, key= lambda x: len(x.dict))
    retu = betterqets, "real"
else:
    retu = qets, "non real"

In [540]:
retu

([Qet({(5, 1, 'c'): 7*sqrt(2)/16, (5, 2, 'c'): sqrt(14)*I/7, (5, 3, 'c'): 5*sqrt(21)/112, (5, 4, 'c'): -sqrt(42)*I/14, (5, 5, 'c'): -3*sqrt(105)/112})],
 'non real')

In [500]:
qetrealsimpler(qets)

KeyboardInterrupt: 

In [495]:
qets = [b for b in bummer if b[0] == 'O_{h}' and b[2] == 4][0][-1]
qetrealsimpler(qets)

([Qet({(4, 2, 'c'): 1}),
  Qet({(4, 0, 'c'): -sqrt(15)/6, (4, 4, 'c'): sqrt(21)/6})],
 'real')

In [496]:
truesol = sp.Matrix(sp.solve(detmat, complement))
sol_symbs = truesol.free_symbols
reality_checks = ([all([sp.im(truesol[0,i].coeff(c)) ==0 for c in sol_symbs]) for i in range(truesol.cols)])
reality_check = all(reality_checks)
if reality_check:
    print("Reality good.")
    while True:
        rep0 = {c:np.random.randint(2) for c in sol_symbs}
        rep1 = {c:np.random.randint(2) for c in sol_symbs}
        vnew0 = truesol.subs(rep0)
        vnew1 = truesol.subs(rep1)
        vnewmat = sp.Matrix([vnew0,vnew1])
        if vnewmat.rank() == 2:
            break
vnewvects = [sp.Matrix(list(vnewmat.row(i))) for i in range(vnewmat.rows)]

Reality good.


In [384]:
truesol

Matrix([[-sqrt(35)*c_2/7, c_1, c_2]])

In [382]:
GramSchmidtAlt(vnewvects)

[Matrix([
 [-sqrt(15)/6],
 [          0],
 [ sqrt(21)/6]]),
 Matrix([
 [0],
 [1],
 [0]])]

In [190]:
def orthosimplify(qet_group):
    # this function takes a list of superposition of Ylm^{sc} and simplifies them
    # qet_group = sym_bases_new['O_{h}'][0][sp.Symbol('E_{g}')][2][0]
    present_basis = list(set(sum([list(qet.dict.keys()) for qet in qet_group],[])))
    if len(present_basis) == len(qet_group):
        new_qet_group = [Qet({(q,):1}) for q in present_basis]
    return new_qet_group

In [283]:
qet_group = sym_bases_new['O_{h}'][0][sp.Symbol('E_{g}')][4][0]
present_basis = list(set(sum([list(qet.dict.keys()) for qet in qet_group],[])))
present_basis = sorted(present_basis, key=lambda x: x[1])
# bases = [set(qet.dict.keys()) for qet in qet_group]
# touching_bases = list(bases[0].intersection(*bases))
# touching_bases = sorted(touching_bases, key=lambda x: x[1])
# pivot = touching_bases[len(touching_bases)//2]
# pivotidx = present_basis.index(pivot)
vs = [qet.vec_in_basis(present_basis) for qet in qet_group]
v1, v2 = vs
realqs = [[sp.im(v)==0 for v in vee] for vee in [v1,v2]]
realq1, realq2 = realqs
realcounts = [rq.count(False) for rq in realqs]
if 1 in realcounts:
    pivot_loc = realcounts.index(1)
else:
    1/0
pivot_idx = realqs[pivot_loc].index(False)



In [284]:
pivot_idx

1

In [277]:
pivot_loc

0

In [269]:
real_qs

[[True, False, True], [False, False, False]]

In [ ]:


finalq1 = Qet({pivot:1})
finalv2 = v2
finalv2[pivotidx] = 0
finalq2 = Qet({b:v for b,v in zip(present_basis,finalv2)})
normalizer = 1/finalq2.norm()
finalq2 = normalizer * finalq2

In [261]:
[sp.im(v)==0 for v in v1]

[True, False, True]

In [254]:
list(map(sp.N,finalq2.dict.values()))

[-0.322748612183951 + 0.559016994374947*I,
 0.381881307912987 - 0.661437827766148*I]

In [246]:
finalq2

Qet({(4, 0, 'c'): 2*sqrt(102)*(-sqrt(30)/24 + sqrt(10)*I/8)/17, (4, 2, 'c'): sqrt(17)*(3 + sqrt(3)*I)/17})

In [218]:
pivot

(4, 2, 'c')

In [298]:
v1

[sqrt(30)/12, sqrt(2)*I/2, -sqrt(42)/12]

In [193]:
orthosimplify(sym_bases_new['O_{h}'][0][sp.Symbol('E_{g}')][2][0])

[Qet({((2, 0, 'c'),): 1}), Qet({((2, 2, 'c'),): 1})]

In [186]:
new_qet_group

[Qet({((2, 0, 'c'),): 1}), Qet({((2, 2, 'c'),): 1})]

In [150]:
[b for b in okay if b[0] == 'O_{h}' and b[2] == 1]

[('O_{h}',
  T_{{1u}},
  1,
  0,
  [Qet({(1, 1, 'c'): -1}), Qet({(1, 1, 's'): 1}), Qet({(1, 0, 'c'): 1})])]

In [103]:
[b for b in bummer if b[0] == 'C_{4}' and b[2] == 1]

[('C_{4}',
  E^1,
  1,
  0,
  [Qet({(1, 1, 'c'): sqrt(2)/2, (1, 1, 's'): sqrt(2)*I/2})]),
 ('C_{4}',
  E^2,
  1,
  0,
  [Qet({(1, 1, 'c'): sqrt(2)/2, (1, 1, 's'): -sqrt(2)*I/2})])]

In [91]:
group_label,irrep_label,l,qet_group_idx,_ = bummer[0]
print(group_label,irrep_label,l)
sym_bases_new[group_label][0][irrep_label][l][qet_group_idx]

C_{4} E^1 1


[Qet({(1, 1, 'c'): sqrt(2)/2, (1, 1, 's'): sqrt(2)*I/2})]

In [90]:
group_label,irrep_label,l,qet_group_idx,_ = bummer[0]
sym_bases_new[group_label][1][irrep_label][l][qet_group_idx]

[Qet({(1, 1): 1})]

In [93]:
sym_bases_new[group_label][0][irrep_label][l]

[[Qet({(1, 1, 'c'): sqrt(2)/2, (1, 1, 's'): sqrt(2)*I/2})]]

In [80]:
bummer = []
okay = []
for group_label, sym_basis in sym_bases_new.items():
    for irrep_label, l_dict in sym_basis[1].items():
        for l, qet_groups in l_dict.items():
            for qet_group_idx, qet_group in enumerate(qet_groups):
                # all_coeffs = sum([list(qet.dict.values()) for qet in qet_group],[])
                reality_check = [real_or_imagined(qet)[0] == 'phased' for qet in qet_group]
                if not all(reality_check):
                    bummer.append((group_label,irrep_label,l,qet_group_idx,qet_group))
                else:
                    okay.append((group_label,irrep_label,l,qet_group_idx,qet_group))

In [85]:
group_label,irrep_label,l,qet_group_idx,_ = bummer[0]
sym_bases_new[group_label][0][irrep_label][l][qet_group_idx]

[Qet({(1, 0, 'c'): sqrt(2)/2, (1, 1, 's'): -sqrt(2)/2})]

In [74]:
print("%d Failed out of %d () (%.1f%%)" % (len(bummer),len(okay)+len(bummer), 100*len(bummer)/(len(okay)+len(bummer))))
failed_group_counts = {}
for bumm in bummer:
    group_label = bumm[0]
    if group_label not in failed_group_counts:
        failed_group_counts[group_label] = 0
    failed_group_counts[group_label] += 1
success_group_counts = {}
for ok in okay:
    group_label = ok[0]
    if group_label not in success_group_counts:
        success_group_counts[group_label] = 0
    success_group_counts[group_label] += 1
for group_label in success_group_counts:
    successful = success_group_counts[group_label]
    total = success_group_counts[group_label]
    if group_label in failed_group_counts:
        total += failed_group_counts[group_label]
        failed = failed_group_counts[group_label]
    else:
        failed = 0
    print("%s: %d failed out of %d" % (group_label, failed, total))

432 Failed out of 2102 () (20.6%)
C_{1}: 0 failed out of 81
C_{i}: 0 failed out of 81
C_{2}: 0 failed out of 81
C_{s}: 0 failed out of 81
C_{2h}: 0 failed out of 81
D_{2}: 0 failed out of 81
C_{2v}: 0 failed out of 81
D_{2h}: 40 failed out of 81
C_{4}: 40 failed out of 81
S_{4}: 0 failed out of 81
C_{4h}: 40 failed out of 81
D_{4}: 0 failed out of 61
C_{4v}: 0 failed out of 61
D_{2d}: 0 failed out of 61
D_{4h}: 0 failed out of 61
C_{3}: 54 failed out of 81
S_{6}: 54 failed out of 81
D_{3}: 0 failed out of 54
C_{3v}: 0 failed out of 54
D_{3d}: 0 failed out of 54
C_{6}: 54 failed out of 81
C_{3h}: 54 failed out of 81
C_{6h}: 54 failed out of 81
D_{6}: 0 failed out of 54
C_{6v}: 0 failed out of 54
D_{3h}: 0 failed out of 54
D_{6h}: 0 failed out of 54
T: 14 failed out of 41
T_{h}: 14 failed out of 41
O: 0 failed out of 34
T_{d}: 7 failed out of 34
O_{h}: 7 failed out of 34


In [288]:
print(20+1)

21


# `symmetry_adapted_basis_real`

<center>
<div style="max-width:800px; word-wrap:break-word; text-align:justify; font-size:100%; margin:50px; border:2px dashed blue; padding: 20px" <br>
This calculation of symmetry adapted spherical harmonics is based on Bradley and Cracknell's "The Mathematical Theory of Symmetry in Solids".

For a given value of $l$ the proyection operator shown below is applied to all the $Y_{l,m}^{s,c}$ $(m=0,\dots,l)$. The rows (i.e. fixed t varying s) then consitute sets of linear combinations of spherical harmonics that transform according to the corresponding irreducible representation.

(Bradley & Cracknell eqn 2.2.2)

$W_{t,s}^{\Gamma_i} = \frac{d_i}{|G|} \sum_{R \in G} D_{t,s}^{\Gamma_i *}(R) R$

where

- $d_i$ =  dimension of the irreducible representation $\Gamma^i$,
- $|G|$ = the order of the group,
- $R$ = the operations from the group,
- $D^i(R)$ the matrix for the operation R in the irreducible representation $\Gamma^i$.

These rows may not be linearly independent, so a bit of linear algebra is needed to determine the largest set of basis that may be generated.

</div>
</center>

## Function foundry

### Assorted RYlm, Wigner_d, Wigner_D, GramSchmidtAlt

In [ ]:
# def RYlm(l, m, op_params):
#     '''
#     Given  a group operation (as parametrized with the
#     iterable  op_params which contains Euler angles α,
#     β,  γ  and  the determinant of the operation) this
#     function  returns  the  effect that this operation
#     has on the sperical harmonic Y_lm.

#     The  result  is  a  qet  whose  keys correspond to
#     values   of   (l,m)   and  whose  values  are  the
#     corresponding coefficients. Keys not present imply
#     that the corresponding coefficient is zero.

#     Examples
#     --------

#     >>> print(RYlm(2,2,(pi, pi/2, pi, 1)))
#         Qet({(2, -2): 1/4,
#              (2, -1): 1/2,
#              (2, 0): sqrt(6)/4,
#              (2, 1): -1/2,
#              (2, 2): 1/4})

#     Reference
#     ---------
#     - Bradley and Cracknell 2.1.3
#     '''
#     if (l,m,*op_params) in RYlm.dict:
#         return RYlm.dict[(l,m,*op_params)]
#     alpha, beta, gamma, detOp = op_params
#     Rf = Qet()
#     for nn in range(-l,l+1):
#         wigD = Wigner_D(l, m, nn, alpha, beta, gamma)
#         if wigD != 0:
#             Rf = Rf + Qet({(l,nn): wigD})
#     ret = (sp.S(detOp)**l) * Rf
#     RYlm.dict[(l,m,*op_params)] = ret
#     return ret
# RYlm.dict = {}

# def Wigner_d(l, n, m, β):
#     '''

#     Reference
#     ---------
#     - Bradley and Cracknell 2.1.6
#     '''
#     k_min = max(0, m-n)
#     k_max = min(l-n, l+m)
#     Wig_d_prefact = sp.sqrt((sp.factorial(l + n)
#                           *sp.factorial(l + m)
#                           *sp.factorial(l - n)
#                           *sp.factorial(l - m)))
#     Wig_d_summands = [((-sp.S(1))**(k - m + n)
#                       * sp.cos(β/2)**(2*l + m - n - 2*k)
#                       * sp.sin(β/2)**(2*k + n - m)
#                       / sp.factorial(l - n - k)
#                       / sp.factorial(l + m - k)
#                       / sp.factorial(k)
#                       / sp.factorial(k - m + n)
#                       )
#                       for k in range(k_min,k_max+1)]
#     Wig_d = (Wig_d_prefact*sum(Wig_d_summands)).doit()
#     return Wig_d

# def Wigner_D(l, n, m, alpha, beta, gamma):
#     '''
#     This is for Euler angles referenced against fixed axes in the z-y-z
#     convention.
    
#     Reference
#     ---------
#     Bradley and Cracknell 2.1.4
#     '''
#     args = (l, n, m, alpha, beta, gamma)
#     if args in Wigner_D.values.keys():
#       return Wigner_D.values[args]
#     if beta == 0:
#       if n == m:
#         Wig_D = (sp.cos(m*alpha + m*gamma) - I * sp.sin(m*alpha + m*gamma))
#       else:
#         Wig_D = 0
#     elif beta == pi:
#       if n == -m:
#         if l % 2 == 0:
#           Wig_D =   (sp.cos(-m*alpha + m*gamma) + I * sp.sin(-m*alpha + m*gamma))
#         else:
#           Wig_D = - (sp.cos(-m*alpha + m*gamma) + I * sp.sin(-m*alpha + m*gamma))
#       else:
#         Wig_D = 0
#     else:
#       Wig_D_0 = I**(abs(n) + n - abs(m) - m) # this always evaluates to a real number
#       Wig_D_1 = (sp.cos(-n*gamma - m*alpha) \
#                  + I * sp.sin(-n * gamma - m * alpha)) * Wigner_d(l,n,m,beta)
#       Wig_D = Wig_D_0 * Wig_D_1
#     Wigner_D.values[args] = Wig_D
#     return Wig_D
# Wigner_D.values = {}

# def GramSchmidtAlt(vs, orthonormal=True):
#     def projection(u,v):
#         return (sp.S(u.dot(v,hermitian=True))/u.dot(u,hermitian=True))*u
#     us = {}
#     us[0] = vs[0]
#     for k in range(1,len(vs)):
#         vk = vs[k]
#         projected_bit = sp.Matrix([0]*len(vs[0]))
#         for j in range(k):
#             projected_bit += sp.simplify(projection(us[j],vk))
#         us[k] = vs[k] - projected_bit
#     if orthonormal:
#         es = [sp.simplify(us[k]/sp.sqrt(sp.S(us[k].dot(sp.conjugate(us[k]))))) for k in us]
#         return es
#     else:
#         return [sp.simplify(us[k]) for k in range(len(us))]

In [8]:
# def basis_check(group_label, irrep_symbol, qets, full_output = False):
#     '''
#     This  function  checks  if  a  list  of qets (which are interpreted as
#     superpositions  of  Ylms  for  fixed  l),  are  a  basis for the given
#     representation.  This  is  done by going through all the operations of
#     the  group,  applying  the  corresponding  rotations  to  all  of  the
#     components  of  the  qets  and  seeing  if  the result matches what is
#     obtained   by   directly   using   the   matrix  for  the  irreducible
#     representation.

#     This check is made against the current irreducible representation, so 
#     that the given qets might still transform as an equivalent irreducible
#     rep.
#     '''
#     group = CPGs.get_group_by_label(group_label)
#     if not isinstance(irrep_symbol, sp.Symbol):
#         irrep_symbol = sp.Symbol(irrep_symbol)
#     if len(qets) != group.irrep_dims[irrep_symbol]:
#         print("A basis needs to have as many entries as the size of the irrep!")
#     irrep_way = []
#     irrep_matrices = group.irrep_matrices[irrep_symbol]
#     irrep_dim = group.irrep_dims[irrep_symbol]
#     all_comparisons = {}
#     all_checks = {}
#     for R, DR in irrep_matrices.items():
#         for idx, qet in enumerate(qets):
#             direct_way = sum([v*RYlm(*k,group.euler_angles[R]) for k,v in qet.dict.items()],Qet())
#             irrep_way = sum([DR[k,idx]*qets[k] for k in range(irrep_dim)],Qet())
#         checks = []
#         if set(irrep_way.dict.keys()) == set(direct_way.dict.keys()):
#             for key in set(irrep_way.dict.keys()):
#                 v1 = sp.N(irrep_way.dict[key],chop=True)
#                 v2 = sp.N(direct_way.dict[key],chop=True)
#                 checks.append(sp.N(v1-v2,chop=True) == 0)
#         else:
#             check = (False)
#         if sum(checks) == len(checks):
#             check = True
#         else:
#             check = False
#         all_comparisons[R] = (check,direct_way, irrep_way)
#         all_checks[R] = check
#     if sum(all_checks.values()) == len(all_checks):
#         all_good = True
#     else:
#         all_good = False
#     if full_output:
#         return all_comparisons
#     else:
#         return all_good

In [2]:
def irrep_check(group, a_basis_list, irrep, l):
    '''
    This  function  checks  if  a  list  of qets (which are interpreted as
    superpositions  of Ylms for fixed l), are a basis for the given irrep.
    This  is done by calculating the matrices that represent the effect of
    applying  all  of  the  group  operations, and then checking to see if
    those matrices are unitary, satisfy the multiplication table, and have
    the corresponding character for the given irrep.

    This function could easily be modifiec to return the matrices that the
    given basis induces.

    Parameters
    ----------

    group    (qdef.CrystalGroup)

    a_basis_list   (list[list[qdef.Qet]]): interpreted  as  superpositions 
        of standard spherical harmonics. Each list must have as many qets
        as the dimension of the corresponding irrep. Several groups may be
        given to account for possible degeneracy of the bases.

    irrep   (sp.Symbol): symbol for one of the group irreps.

    l      (int): the  value  of  l  from which the given qets are seen to
        originate from.
    
    Returns
    -------
    all_checks (list): a list of nested lists, of which there are as many
    as groups of qets were given. The enclosed lists have three Bool, the
    first one of which is the check for the multiplication table, the 2nd
    of which is the of characters of the induced matrices, and the  third
    being the check against the matrices being unitary.
    '''
    all_checks = []
    for a_basis in a_basis_list:
        lbasis = [(l,m) for m in range(-l,l+1)]
        basis_matrix = sp.Matrix([b.vec_in_basis(lbasis) for b in a_basis]).T
        basis_matrix = Dagger(basis_matrix)
        induced_matrices = {}
        char_checks = {}
        unitarity_checks = {}
        for group_op, op_params in group.euler_angles.items():
            D_matrix = (op_params[3]**l)*sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)]).T
            R_rows = []
            for basis_element in a_basis:
                vector = sp.Matrix(basis_element.vec_in_basis(lbasis))
                t_vector = D_matrix*vector
                row = (basis_matrix*t_vector).T
                R_rows.append(row)
            induced_matrices[group_op] = sp.Matrix(R_rows).T
            induced_matrices[group_op] = sp.re(induced_matrices[group_op]) + sp.I * sp.im(induced_matrices[group_op])
            diff = sp.N(sp.simplify(induced_matrices[group_op]).trace() - sp.simplify(group.irrep_matrices[irrep][group_op]).trace(), chop=True)
            char_check = (diff == 0)
            if not char_check:
                warnings.warn("Character mismatch.")
            char_checks[group_op] = char_check
            ru = sp.simplify(induced_matrices[group_op] * Dagger(induced_matrices[group_op]))
            ru = sp.re(ru) + sp.I*sp.im(ru)
            unitarity_checks[group_op] =  (ru == sp.eye(group.irrep_dims[irrep]) )
        # check to see if the product table is satisfied
        ptable = group.multiplication_table_dict
        checks = {}
        for group_op_0 in group.group_operations:
            for group_op_1 in group.group_operations:
                group_op_01 = ptable[(group_op_0,group_op_1)]
                target = induced_matrices[group_op_01]
                directp = sp.simplify(induced_matrices[group_op_0]*induced_matrices[group_op_1])
                directp = sp.re(directp) + sp.I * sp.im(directp)
                target = sp.re(target) + sp.I * sp.im(target)
                pcheck = sp.N(directp - target, chop=True) == sp.zeros(target.rows)
                if not pcheck:
                    warnings.warn("Multiplication table mismatch.")
                checks[(group_op_0,group_op_1)] = pcheck
        all_checks.append((all(checks.values()), all(char_checks.values()), all(unitarity_checks.values())))
    return all_checks

### symmetry_adapted_basis

In [3]:
# def real_or_imagined(qet):
#     '''
#     For  a given superposition of spherical harmonics, determine
#     if the total is just shy from being real except for a global
#     complex phase.

#     Parameters
#     ----------
#     qet (qdefcore.Qet)

#     Returns
#     -------
#     (3-tuple) qet_type, final_qet, phase
#      If qet_type == 'phased' then the returned qet is real, and the
#      phase is the global phase that was divided out.
#      If qet_type == 'mixed' the returned qet is the original one, and 
#      phase is equal to None.

#     '''
#     lvalues = list(set([x[0] for x in qet.dict.keys()]))
#     assert len(lvalues) == 1, "Only valid for a single value of l."
#     qet_l = lvalues[0]
#     mvalues = [x[1] for x in qet.dict.keys()]
#     mabs = set(map(abs, mvalues))
#     msymkeys = all([mab in mvalues and -mab in mvalues for mab in mabs])
#     if not msymkeys:
#         warnings.warn("Mixed qet.")
#         return 'mixed', qet, None
#     else:
#         # valences = []
#         hashes = []
#         for mab in mabs:
#             minus_coeff = qet.dict[(qet_l, -mab)]
#             plus_coeff = qet.dict[(qet_l, mab)]
#             if abs(minus_coeff) != abs(plus_coeff):
#                 return 'mixed', qet, None
#             else:
#                 if mab % 2 == 0:
#                     phaser_sign = 1
#                 else:
#                     phaser_sign = -1
#                 hashed = plus_coeff*(1+mab*sp.I) + phaser_sign*minus_coeff*(1-mab*sp.I)
#                 hashes.append(hashed)
#     # try dividing all the hashes by the first one
#     pivot_hash = sp.simplify(hashes[0])
#     pivot_hash = abs(sp.re(pivot_hash)) + sp.I*abs(sp.im(pivot_hash))
#     # see if all of them are real
#     pencil = [sp.im(hashed/pivot_hash) == 0 for hashed in hashes]
#     if all(pencil):
#         # the phase shouldn't modify the norm
#         pivot_hash = pivot_hash/abs(pivot_hash)
#         pivot_hash = sp.simplify(pivot_hash)
#         if sp.im(pivot_hash) == 0:
#             return 'phased', qet, 1
#         final_qet = (sp.S(1)/pivot_hash) * qet
#         return 'phased', final_qet, pivot_hash
#     else:
#         warnings.warn("Mixed qet.")
#         return 'mixed', qet, None

# def real_or_imagined_global_unitary(qets):
#     '''
#     For a set of qets, this function checks to see if all of them
#     can be brought to be real by a set of unimodular complex numbers.
#     If this is the case, then the resulting set of qets transform
#     as an equivalent irreducible representation.
#     '''
#     if len(qets) == 0:
#         return qets, None, None
#     phased_qets = list(map(real_or_imagined, qets))
#     all_phases = list(map(lambda x: x[-1], phased_qets))
#     phases = set(list(map(lambda x: x[-1], phased_qets)))
#     if None in phases:
#         warnings.warn("no global acceptable phase found.")
#         return qets, 0, None
#     elif all(list(map(lambda x: abs(x) == 1,phases))) or len(phases) == 1:
#         return list(map(lambda x: x[1], phased_qets)), 1, all_phases
#     else:
#         warnings.warn("no acceptable diagonal unitary transform found")
#         return qets, 2, all_phases

In [7]:
def symmetry_adapted_basis_v_real(group_label, lmax, verbose=False):
    '''
    Starting from a set of functions it may be possible to construct a new
    set which transforms according to the irreducible representations of a
    given  group.  This  is  done by using the proyection operators of the
    group.  Given  that  these  operators  may  produce linearly dependent
    combinations  some  amount of linear algebra is necessary to determine
    the largest set of linearly independent sets of functions.

    A  necessary  component  of  the  algorithm  is  knowledge  of how the
    starting  functions  transform  under  the group's operations. This is
    well  known  for  the  spherical  harmonics  in  terms of the Widger D
    matrices.

    It  may  also  be  desirable to have the resulting set of functions be
    real,  in  which case it is more convenient to start with the sine and
    cosine combinations of spherical harmonics. In this function this path
    is taken.

    After  a  set  has  been  found,  another  set,  which would transform
    accordingly  to an equivalent irreducible representation, may be found
    by applying a unitary transformation.  This  is the effect of applying
    the function real_or_imagined_global_unitary somewhere near the end of
    this function.

    Parameters
    ----------
    group_label    (str): a label for a crystallographic point group
    lmax           (int): up  to  which  value  of the l the bases will be 
                          constructed.

    Returns
    -------
    (2-tuple)  symmetry_basis_sc,  symmetry_basis: where symmetry_basis_sc
    is  the  resultant  basis  in  terms  of  sine  and  cosine  spherical
    harmonics,  and  symmetry_basis  is  for  the  result  translated into
    standard spherical harmonics.
    Both  these are nested dictionaries where the first key corresponds to
    labels  (sp.Symbol)  of  the irreducible representations of the group,
    whose  second  key  corresponds  to a value of l, and whose value is a
    list  of  lists where each list if of length equal to the dimension of
    the  irreducible  representation  (there  may   be  more than one list
    because  at  times  there's  more  than one possible basis that may be
    formed).
    '''
    GramSchmidtFun = GramSchmidtAlt
    group = CPGs.get_group_by_label(group_label)
    group_irreps = group.irrep_labels
    symmetry_basis_sc = {}
    symmetry_basis = {}
    
    for group_irrep in group_irreps:
        if verbose:
            print(str(group_irrep))
        irrep_dim = group.irrep_dims[group_irrep]
        symmetry_basis_sc[group_irrep] = {}
        symmetry_basis[group_irrep] = {}
        irrep_matrices = group.irrep_matrices[group_irrep]
    
        for l in range(lmax+1):
            full_basis = [(l,m) for m in range(-l,l+1)]
            full_basis_real = [(l,m,'c') for m in range(0,l+1)] + [(l,m,'s') for m in range(1,l+1)]
            realharm_qet_basis = [Qet({(l,m,'c'):1}) for m in range(0,l+1)] + [Qet({(l,m,'s'):1}) for m in range(1,l+1)]
            standard_qet_basis = [Qet({(l,m):1}) for m in range(-l,l+1)]

            # We need two change of coordinates matrices that allow going
            # back and forth from the Y_lm^{cs} to the Y_lm.
            
            combos = [Qet({
                (l,abs(m),'c'):sp.S(1)/sp.sqrt(2),
                (l,abs(m),'s'):-sp.I/sp.sqrt(2)}).vec_in_basis(full_basis_real) for m in range(-l,0)]
            combos.append(Qet({(l,0,'c'):1}).vec_in_basis(full_basis_real))
            combos = combos + [Qet({
                (l,m,'c'): sp.S(1)/sp.sqrt(2),
                (l,m,'s'): sp.I*sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis_real) for m in range(1,l+1)]

            # When this matrix multiplies a vector of coefficients of regular spherical harmonics
            # the results are the coefficients in the basis composed of Y_lm^{sc}

            change_of_basis_matrix = sp.Matrix(combos).T # this matrix changes from the regular Ylm to Ylm^cs

            combos2 = [Qet({(l,0):1}).vec_in_basis(full_basis)]
            combos2 = combos2 + [Qet({
                (l, m): sp.S(1)/sp.sqrt(2),
                (l,-m): 1/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
            combos2 = combos2 + [Qet({
                (l,m) : -sp.I/sp.sqrt(2),
                (l,-m): sp.I/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]

            # When this matrix multiplies a vector of coefficients of Y_lm^{sc}
            # the results are the coefficients in the basis composed of standard spherical harmonics.

            change_of_basis_matrix_from_real_to_standard = sp.Matrix(combos2).T
            
            # This loop applied the proyection operators and creates a dictionary
            # all_phis whose key is the value of m that relates to it
            # and whose valuees are dictionaries whose keys are 3-tuples (t,s,'s' or 'c')

            all_phis = {}
            for m in range(0,l+1):
                phis = {}
                # for a fixed row t,
                for t in range(irrep_dim):
                    # collect of of the sums by adding over columns
                    cphi = Qet({})
                    sphi = Qet({})
                    for s in range(irrep_dim):
                        for R, DR in irrep_matrices.items():
                            dr = sp.conjugate(DR[t,s])
                            op_params = group.euler_angles[R]
                            if dr != 0:
                                if m == 0:
                                    cphi = cphi + dr*RYlm(l,m,op_params)
                                else:
                                    cphi = cphi + (1/sp.sqrt(2)) * dr    * RYlm(l,m,op_params)
                                    cphi = cphi + (1/sp.sqrt(2)) * dr    * RYlm(l,-m,op_params)
                                    sphi = sphi + (sp.I/sp.sqrt(2)) * dr * RYlm(l,m,op_params)
                                    sphi = sphi - (sp.I/sp.sqrt(2)) * dr * RYlm(l,-m,op_params)

                        coord_vec = sp.Matrix(cphi.vec_in_basis(full_basis))
                        transformed_vec = list(change_of_basis_matrix*coord_vec)
                        transformed_qet = sum([v*q for v,q in zip(transformed_vec, realharm_qet_basis) if v !=0], Qet({}))
                        phis[(t,s,'c')] = (sp.S(irrep_dim)/group.order)*transformed_qet
                        if m!=0:
                            coord_vec_sin = sp.Matrix(sphi.vec_in_basis(full_basis))
                            transformed_vec = list(change_of_basis_matrix*coord_vec_sin)
                            transformed_qet = sum([v*q for v,q in zip(transformed_vec, realharm_qet_basis) if v !=0],Qet({}))
                            phis[(t,s,'s')] = (sp.S(irrep_dim)/group.order)*transformed_qet
                all_phis[m] = phis

            # Take the qets and find the coefficients in the basis full_basis
            # this is necessary to evaluate linear independence, and useful
            # for applying the Gram-Schmidt orthonormalization process.
            coord_vecs = []
            for m,s,t in product(range(0,l+1),range(irrep_dim),range(irrep_dim)):
                coord_vecs.append(all_phis[m][(t,s,'c')].vec_in_basis(full_basis_real))
            # This second loop seems as it could be integrated in the one above
            # but this is not so, the order in which rows are added matters.
            for m,s,t in product(range(0,l+1),range(irrep_dim),range(irrep_dim)):
                if m != 0:
                    coord_vecs.append(all_phis[m][(t,s,'s')].vec_in_basis(full_basis_real))
            if verbose:
                print("Constructing a big matrix with coordinates in standard basis....")
            bigmatrix = sp.Matrix(coord_vecs)
            num_lin_indep_rows = (bigmatrix.rank())
            good_rows = []
            if num_lin_indep_rows != 0:
                if verbose:
                    print("There are %d linearly independent entries..." % (num_lin_indep_rows))
                    print("Collecting that many, and in groups of %d, from the original set..." % irrep_dim)
                assert (bigmatrix.rows % irrep_dim) == 0, "No. of indep entries should divide the dim of the irrep."
                # determine what groupings have the full rank of the matrix
                chunks = [(coord_vecs[i*irrep_dim : (i*irrep_dim + irrep_dim)]) for i in range(bigmatrix.rows // irrep_dim)]
                cycles = num_lin_indep_rows // irrep_dim
                chunks = [chunk for chunk in chunks if (sp.Matrix(chunk).rank() == irrep_dim)]
                for bits in combinations(chunks,cycles):
                    mbits = list(map(lambda x: [sp.Matrix(x)], bits))
                    tot = sp.Matrix(sp.BlockMatrix(mbits))
                    if tot.rank() == num_lin_indep_rows:
                        # a satisfactory subset has been found, exit
                        break
                else:
                    raise Exception("Couldn't find an adequate subset of rows.")
                for bit in bits:
                    good_rows.append(bit)
                if verbose:
                    print("Orthonormalizing ...")
                flat_rows = []
                degeneracy = len(good_rows)
                for rows in good_rows:
                    flat_rows.extend(rows)
                flat_rows = list(map(sp.Matrix,flat_rows))
                normalized = GramSchmidtFun(flat_rows, orthonormal=True)
                parts = []
                for deg in range(degeneracy):
                    chunk = list(map(list,normalized[deg*irrep_dim : deg*irrep_dim + irrep_dim]))
                    parts.append(chunk)
                all_normal_qets = []
                all_regular_qets = []
                # convert the coefficient vectors back to qets
                skip_check = False
                all_phases = []
                for part in parts:
                    normal_qets = [Qet({k: v for k,v in zip(full_basis_real,part[i]) if v!=0}) for i in range(len(part))]
                    spherical_qets = []
                    for nq in normal_qets:
                        coord_vec = sp.Matrix(nq.vec_in_basis(full_basis_real))
                        transformed_vec = list(change_of_basis_matrix_from_real_to_standard*coord_vec)
                        transformed_qet = sum([v*q for v,q in zip(transformed_vec, standard_qet_basis) if v !=0],Qet({}))
                        spherical_qets.append(transformed_qet)
                    all_normal_qets.append(normal_qets)
                    phased, flag, phases = real_or_imagined_global_unitary(spherical_qets)
                    all_phases.append(phases)
                    if flag == 0:
                        skip_check = True
                    all_regular_qets.append(phased)
                if not skip_check:
                    num_different_phase_shift = len(set(tuple(map(tuple, all_phases))))
                    if num_different_phase_shift != 1:
                        warnings.warn("Phases between degenerate bases are incompatible.")
                if verbose:
                    print("Finished!")
                symmetry_basis_sc[group_irrep][l] = all_normal_qets
                symmetry_basis[group_irrep][l] = all_regular_qets
            else:
                symmetry_basis_sc[group_irrep][l] = []
                symmetry_basis[group_irrep][l] = []
    return symmetry_basis_sc, symmetry_basis

In [5]:
# for group_label in CPGs.all_group_labels:
#     print(group_label)
#     symmetry_adapted_basis_v_real(group_label,2)

In [ ]:
# all_coeffs = []
# for qets in abasis[0][sp.Symbol('A_1')][9]:
#     print("*******")
#     qet_coeffs = []
#     for qet in qets:
#         coeffs = []
#         print("------")
#         for k,v in qet.dict.items():
#             print('(%s,%s): %f' % (k[1],k[2],float(v)))
#             coeffs.append(v)
#         qet_coeffs.append(coeffs)
#     all_coeffs.append(qet_coeffs)
# old_symmetry_bases = pickle.load(open('/Users/juan/Zia Lab/Codebase/qdef/data/old_symmetry_bases.pkl','rb'))
# all_coeffs = []
# for qets in old_symmetry_bases['O'][sp.Symbol('T_2')][8]:
#     print("*******")
#     qet_coeffs = []
#     for qet in qets:
#         coeffs = []
#         print("------")
#         for k,v in qet.dict.items():
#             print('(%s): %f' % (k[1],sp.sqrt(1/2.)*abs(v)))
#             coeffs.append(v)
#         qet_coeffs.append(coeffs)
#     all_coeffs.append(qet_coeffs)

## Calculating all, exporting, and checking for orthonormality 

(Last run on Dec-22 2021-12-22 20:13:01)

NOTE: After exporting to pkl, remove CPGs.pkl, shutdown kernel, re-import qdef.

In [8]:
# turn it up all the way to 8
symmetry_adapted_basis = symmetry_adapted_basis_v_real
lmax = 2
sym_bases = Parallel(n_jobs = num_cores)(delayed(symmetry_adapted_basis)(group_label, lmax) for group_label in CPGs.all_group_labels)
all_the_symmetry_bases = dict(zip(CPGs.all_group_labels,sym_bases))
!beep

In [10]:
from sympy.physics.quantum.dagger import Dagger

In [11]:
total_checks = {}
all_checks = []
for group_label, basis_dict in all_the_symmetry_bases.items():
    print(group_label)
    group = CPGs.get_group_by_label(group_label)
    total_checks[group_label] = {}
    for irrep_symbol, l_dict in basis_dict[1].items():
        total_checks[group_label][irrep_symbol] = []
        for l, qet_groups in l_dict.items():
            checking = sum(irrep_check(group, qet_groups, irrep_symbol, l),())
            total_checks[group_label][irrep_symbol].append(checking)
            all_checks.append(all(checking))

C_{1}
C_{i}
C_{2}
C_{s}
C_{2h}
D_{2}
C_{2v}
D_{2h}
C_{4}
S_{4}
C_{4h}
D_{4}
C_{4v}
D_{2d}
D_{4h}
C_{3}
S_{6}
D_{3}
C_{3v}
D_{3d}
C_{6}
C_{3h}
C_{6h}
D_{6}
C_{6v}
D_{3h}
D_{6h}
T
T_{h}
O
T_{d}
O_{h}
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [ ]:
def echo(*args):
    return args
checklist_args = Parallel(n_jobs = num_cores)(delayed(echo)(group_label, irrep_symb, basis) \
                                        for group_label in CPGs.all_group_labels\
                                        for irrep_symb in CPGs.get_group_by_label(group_label).irrep_labels \
                                        for l in range(lmax) \
                                        for (basis_idx, basis) in enumerate(all_the_symmetry_bases[group_label][1][irrep_symb][l])
                                        )

A complete basis needs to satisfy the following:

- For a fixed irreducible representation, for all of the group operations the components of each basis needs to transform accordingly to the irreducible representation that corresponds to the group operation.

- Within a basis, all of the components are orthonormal.

- If there's more than one basis, then all of the components of the one are also orthogonal to the components of the other.

- For a given value of l, if one collects all of the components for all of the irreducible representations, there are a total of (2l+1) of them.

In [ ]:
checklist = Parallel(n_jobs = num_cores)(delayed(basis_check)(group_label, irrep_symb, basis) \
                                        for group_label in CPGs.all_group_labels\
                                        for irrep_symb in CPGs.get_group_by_label(group_label).irrep_labels \
                                        for l in range(lmax) \
                                        for (basis_idx, basis) in enumerate(all_the_symmetry_bases[group_label][1][irrep_symb][l])
                                        )

In [ ]:
if len(checklist) == sum(checklist):
    print("Good tidings!")
    print("Out of %d instances %d passed the test." % (len(checklist), sum(checklist)))
else:
    print("Out of %d instances %d passed the test." % (len(checklist), sum(checklist)))

In [ ]:
!beep

In [ ]:
l = 2
irrep = sp.Symbol('T_2')
a_basis = all_the_symmetry_bases['O'][1][sp.Symbol('T_2')][l][0]
group = CPGs.get_group_by_label('O')
irreps = group.irrep_labels
lbasis = [(l,m) for m in range(-l,l+1)]
basis_matrix = sp.Matrix([b.vec_in_basis(lbasis) for b in a_basis]).T
basis_matrix = Dagger(basis_matrix)
induced_matrices = {}
char_checks = {}
unitarity_checks = {}
for group_op, op_params in group.euler_angles.items():
    D_matrix = sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)])
    R_rows = []
    for basis_element in a_basis:
        vector = sp.Matrix(basis_element.vec_in_basis(lbasis))
        t_vector = D_matrix*vector
        row = (basis_matrix*t_vector).T
        R_rows.append(row)
    induced_matrices[group_op] = sp.Matrix(R_rows).T
    char_checks[group_op] = induced_matrices[group_op].trace() == group.irrep_matrices[irrep][group_op].trace()
    unitarity_checks[group_op] = sp.simplify(induced_matrices[group_op] * Dagger(induced_matrices[group_op])) == sp.eye(group.irrep_dims[irrep])
# check to see if the product table is satisfied
ptable = group.multiplication_table_dict
checks = {}
for group_op_0 in group.group_operations:
    for group_op_1 in group.group_operations:
        group_op_01 = ptable[(group_op_1,group_op_0)]
        target = induced_matrices[group_op_01]
        directp = sp.simplify(induced_matrices[group_op_0]*induced_matrices[group_op_1])
        checks[(group_op_0,group_op_1)] = (directp == target)
print(all(checks.values()), all(char_checks.values()))

In [ ]:
# def irrep_check(group_label, a_basis_list, irrep, l):
#     all_checks = []
#     for a_basis in a_basis_list:
#         group = CPGs.get_group_by_label(group_label)
#         lbasis = [(l,m) for m in range(-l,l+1)]
#         basis_matrix = sp.Matrix([b.vec_in_basis(lbasis) for b in a_basis]).T
#         basis_matrix = Dagger(basis_matrix)
#         induced_matrices = {}
#         char_checks = {}
#         unitarity_checks = {}
#         for group_op, op_params in group.euler_angles.items():
#             D_matrix = sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)])
#             R_rows = []
#             for basis_element in a_basis:
#                 vector = sp.Matrix(basis_element.vec_in_basis(lbasis))
#                 t_vector = D_matrix*vector
#                 row = (basis_matrix*t_vector).T
#                 R_rows.append(row)
#             induced_matrices[group_op] = sp.Matrix(R_rows).T
#             char_checks[group_op] = induced_matrices[group_op].trace() == group.irrep_matrices[irrep][group_op].trace()
#             unitarity_checks[group_op] = sp.simplify(induced_matrices[group_op] * Dagger(induced_matrices[group_op])) == sp.eye(group.irrep_dims[irrep])
#         # check to see if the product table is satisfied
#         ptable = group.multiplication_table_dict
#         checks = {}
#         for group_op_0 in group.group_operations:
#             for group_op_1 in group.group_operations:
#                 group_op_01 = ptable[(group_op_1,group_op_0)]
#                 target = induced_matrices[group_op_01]
#                 directp = sp.simplify(induced_matrices[group_op_0]*induced_matrices[group_op_1])
#                 checks[(group_op_0,group_op_1)] = (directp == target)
#         all_checks.append((all(checks.values()), all(char_checks.values()), all(unitarity_checks.values())))
#     return all_checks

In [ ]:
# def irrep_check(group, a_basis_list, irrep, l):
#     all_checks = []
#     for a_basis in a_basis_list:
#         lbasis = [(l,m) for m in range(-l,l+1)]
#         basis_matrix = sp.Matrix([b.vec_in_basis(lbasis) for b in a_basis]).T
#         basis_matrix = Dagger(basis_matrix)
#         induced_matrices = {}
#         char_checks = {}
#         unitarity_checks = {}
#         for group_op, op_params in group.euler_angles.items():
#             D_matrix = (op_params[3]**l)*sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)]).T
#             R_rows = []
#             for basis_element in a_basis:
#                 vector = sp.Matrix(basis_element.vec_in_basis(lbasis))
#                 t_vector = D_matrix*vector
#                 row = (basis_matrix*t_vector).T
#                 R_rows.append(row)
#             induced_matrices[group_op] = sp.Matrix(R_rows).T
#             induced_matrices[group_op] = sp.re(induced_matrices[group_op]) + sp.I * sp.im(induced_matrices[group_op])
#             diff = sp.N(sp.simplify(induced_matrices[group_op]).trace() - sp.simplify(group.irrep_matrices[irrep][group_op]).trace(), chop=True)
#             char_check = (diff == 0)
#             if not char_check:
#                 print("++++++++++")
#                 display(diff)
#                 display(induced_matrices[group_op])
#                 display(group.irrep_matrices[irrep][group_op])
#                 display(a_basis)
#                 display(D_matrix)
#                 display(group_op)
#                 display(op_params)
#                 display(basis_matrix)
#             char_checks[group_op] = char_check
#             ru = sp.simplify(induced_matrices[group_op] * Dagger(induced_matrices[group_op]))
#             ru = sp.re(ru) + sp.I*sp.im(ru)
#             unitarity_checks[group_op] =  (ru == sp.eye(group.irrep_dims[irrep]) )
#         # check to see if the product table is satisfied
#         ptable = group.multiplication_table_dict
#         checks = {}
#         for group_op_0 in group.group_operations:
#             for group_op_1 in group.group_operations:
#                 group_op_01 = ptable[(group_op_0,group_op_1)]
#                 target = induced_matrices[group_op_01]
#                 directp = sp.simplify(induced_matrices[group_op_0]*induced_matrices[group_op_1])
#                 directp = sp.re(directp) + sp.I * sp.im(directp)
#                 target = sp.re(target) + sp.I * sp.im(target)
#                 pcheck = sp.N(directp - target, chop=True) == sp.zeros(target.rows)
#                 if not pcheck:
#                     display(directp)
#                     display(target)
#                     display(sp.N(directp - target, chop=True))
#                 checks[(group_op_0,group_op_1)] = pcheck
#         all_checks.append((all(checks.values()), all(char_checks.values()), all(unitarity_checks.values())))
#     return all_checks

In [ ]:
# for group_op, op_params in group.euler_angles.items():
#     D_matrix = sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)])
#     break

In [ ]:
# def irrep_check(group, a_basis_list, irrep, l):
#     all_checks = []
#     for a_basis in a_basis_list:
#         lbasis = [(l,m) for m in range(-l,l+1)]
#         basis_matrix = sp.Matrix([b.vec_in_basis(lbasis) for b in a_basis]).T
#         basis_matrix = Dagger(basis_matrix)
#         induced_matrices = {}
#         char_checks = {}
#         unitarity_checks = {}
#         for group_op, op_params in group.euler_angles.items():
#             D_matrix = (op_params[3]**l)*sp.Matrix([[Wigner_D(l,m,n,*op_params[:3]) for n in range(-l,l+1)] for m in range(-l,l+1)]).T
#             R_rows = []
#             for basis_element in a_basis:
#                 vector = sp.Matrix(basis_element.vec_in_basis(lbasis))
#                 t_vector = D_matrix*vector
#                 row = (basis_matrix*t_vector).T
#                 R_rows.append(row)
#             induced_matrices[group_op] = sp.Matrix(R_rows).T
#             induced_matrices[group_op] = sp.re(induced_matrices[group_op]) + sp.I * sp.im(induced_matrices[group_op])
#             diff = sp.N(sp.simplify(induced_matrices[group_op]).trace() - sp.simplify(group.irrep_matrices[irrep][group_op]).trace(), chop=True)
#             char_check = (diff == 0)
#             if not char_check:
#                 print("++++++++++")
#                 display(diff)
#                 display(induced_matrices[group_op])
#                 display(group.irrep_matrices[irrep][group_op])
#                 display(a_basis)
#                 display(D_matrix)
#                 display(group_op)
#                 display(op_params)
#                 display(basis_matrix)
#             char_checks[group_op] = char_check
#             ru = sp.simplify(induced_matrices[group_op] * Dagger(induced_matrices[group_op]))
#             ru = sp.re(ru) + sp.I*sp.im(ru)
#             unitarity_checks[group_op] =  (ru == sp.eye(group.irrep_dims[irrep]) )
#         # check to see if the product table is satisfied
#         ptable = group.multiplication_table_dict
#         checks = {}
#         for group_op_0 in group.group_operations:
#             for group_op_1 in group.group_operations:
#                 group_op_01 = ptable[(group_op_0,group_op_1)]
#                 target = induced_matrices[group_op_01]
#                 directp = sp.simplify(induced_matrices[group_op_0]*induced_matrices[group_op_1])
#                 directp = sp.re(directp) + sp.I * sp.im(directp)
#                 target = sp.re(target) + sp.I * sp.im(target)
#                 pcheck = sp.N(directp - target, chop=True) == sp.zeros(target.rows)
#                 if not pcheck:
#                     display(directp)
#                     display(target)
#                     display(sp.N(directp - target, chop=True))
#                 checks[(group_op_0,group_op_1)] = pcheck
#         all_checks.append((all(checks.values()), all(char_checks.values()), all(unitarity_checks.values())))
#     return all_checks

In [ ]:
# total_checks = {}
# all_checks = []
# for group_label, basis_dict in all_the_symmetry_bases.items():
#     print(group_label)
#     group = CPGs.get_group_by_label(group_label)
#     total_checks[group_label] = {}
#     for irrep_symbol, l_dict in basis_dict[1].items():
#         total_checks[group_label][irrep_symbol] = []
#         for l, qet_groups in l_dict.items():
#             checking = sum(irrep_check(group, qet_groups, irrep_symbol, l),())
#             total_checks[group_label][irrep_symbol].append(checking)
#             all_checks.append(all(checking))
# print(all_checks)

In [ ]:
irrep_check(group, qet_groups, irrep_symbol, l)

In [ ]:
!beep

In [ ]:
total_checks = {}
for group_label, basis_dict in all_the_symmetry_bases.items():
    print(group_label)
    group = CPGs.get_group_by_label(group_label)
    total_checks[group_label] = {}
    for irrep_symbol, l_dict in basis_dict[1].items():
        total_checks[group_label][irrep_symbol] = []
        for l, qet_groups in l_dict.items():
            checking = irrep_check(group, qet_groups, irrep_symbol, l)
            total_checks[group_label][irrep_symbol].append(checking)

In [ ]:
flat_checks = {}
for group_label, basis_dict in total_checks.items():
    flat_checks[group_label] = []
    for irrep_symbol, l_dict in basis_dict[1].items():
        flat_checks[irrep_symbol].append(total_checks[group_label][irrep_symbol])

In [ ]:
flat_checks

In [ ]:
for check in checks:
    print(check)
    

In [ ]:
dir(group)

In [ ]:
# def real_or_imagined_global_unitary(qets):
#     '''
#     '''
#     if len(qets) == 0:
#         return qets
#     phased_qets = list(map(real_or_imagined, qets))
#     phases = set(list(map(lambda x: x[-1], phased_qets)))
#     if None in phases:
#         warnings.warn("no global acceptable phase found for")
#         return qets, 0
#     elif all(list(map(lambda x: abs(x) == 1,phases))) or len(phases) == 1:
#         return list(map(lambda x: x[1], phased_qets)), 1
#     else:
#         warnings.warn("no acceptable diagonal unitary transform found")
#         return qets, 2

In [ ]:
all_the_symmetry_bases_real = {}
clunky_ones = []
for group_label, bases in all_the_symmetry_bases.items():
    print(group_label)
    all_the_symmetry_bases_real[group_label] = {}
    for irrep, l_dict in bases.items():
        # print(irrep)
        all_the_symmetry_bases_real[group_label][irrep] = {}
        for l, qet_groups in l_dict.items():
            # print(l)
            all_the_symmetry_bases_real[group_label][irrep][l] = []
            for qet_group in qet_groups:
                realized, flag = real_or_imagined_global_unitary(qet_group)
                if flag in [0,2]:
                    clunky_ones.append((group_label, irrep,l,qet_group))
                all_the_symmetry_bases_real[group_label][irrep][l].append(realized)

In [ ]:
clunky_ones[1]

In [ ]:
all_the_symmetry_bases['O'][sp.Symbol('T_1')][1]

In [ ]:
all_the_symmetry_bases_real['O'][sp.Symbol('T_1')][1]

In [ ]:
[checklist_args[idx] for idx in range(len(checklist)) if checklist[idx] == False]

In [ ]:
pickle.dump(all_the_symmetry_bases, open('./data/symmetry_bases.pkl','wb'))

In [ ]:
# See differences between this version and the previous one

In [ ]:
import pickle

In [ ]:
all_the_symmetry_bases_old = pickle.load(open('./data/old_symmetry_bases.pkl','rb'))

In [ ]:
for irrep in all_the_symmetry_bases_old['O']:
    for l in range(2,3):
        old_one = all_the_symmetry_bases_old['O'][irrep][l]
        new_one = all_the_symmetry_bases['O'][irrep][l]
        jackpot = (irrep == sp.Symbol('E'))
        if not (len(str(old_one)) == len(str(new_one))) or True:
            print("-------------------")
            print(l, irrep)
            print("old",old_one)
            print("**********")
            print("new",new_one)

In [ ]:
(all_the_symmetry_bases['O'][sp.Symbol('T_2')][2][0])

## Orthonormality Check (Visual)

In [ ]:
# all_the_symmetry_bases = symmetry_bases

In [ ]:
lmax = 8
base_matrices = {}
for group_label in CPGs.all_group_labels:
#     group = CPGs.get_group_by_label(group_label)
    sym_bases = all_the_symmetry_bases[group_label]
    group_irreps = list(sym_bases.keys())
    these_matrices = {}
    for l in range(lmax+1):
        this_matrix = []
        lbase = [(l,m) for m in range(-l,l+1)]
        for irrep in group_irreps:
            for g in sym_bases[irrep][l]:
                for chunk in g:
                    this_matrix.append(chunk.vec_in_basis(lbase))
        these_matrices[l] = sp.Matrix(this_matrix).T
    base_matrices[group_label] = these_matrices

In [ ]:
lmax = 8
for group_label in CPGs.all_group_labels:
    fig, axes = plt.subplots(nrows=2, ncols=4,figsize=(9,6))
    for l in range(1,lmax+1):
        ax = axes[(l-1)//4,(l-1)%4]
        basis = base_matrices[group_label][l]
        basis_test = np.abs(np.array(sp.physics.quantum.Dagger(basis) * basis).astype(np.cdouble))
        ax.imshow(basis_test, vmin=0, cmap = 'hot', interpolation=None)
        ax.set_title('l=%d' % l)
    fig.suptitle(r'$%s$' % group_label)
    plt.tight_layout()
    plt.show()
    break

### Seeing if the flattened basis is orthogonal

In [ ]:
lmax = 8
base_matrices = {}
for group_label in CPGs.all_group_labels:
    group = CPGs.get_group_by_label(group_label)
    sym_bases = group.symmetry_adapted_bases
    group_irreps = group.irrep_labels
    these_matrices = {}
    for l in range(lmax+1):
        this_matrix = []
        lbase = [(l,m) for m in range(-l,l+1)]
        for irrep in group_irreps:
            for g in sym_bases[irrep][l]:
                for chunk in g:
                    this_matrix.append(chunk.vec_in_basis(lbase))
        these_matrices[l] = sp.Matrix(this_matrix).T
    base_matrices[group_label] = these_matrices

In [ ]:
lmax = 8
for group_label in CPGs.all_group_labels:
    fig, axes = plt.subplots(nrows=2, ncols=4,figsize=(9,6))
    for l in range(1,lmax+1):
        ax = axes[(l-1)//4,(l-1)%4]
        basis = base_matrices[group_label][l]
        basis_test = np.abs(np.array(sp.physics.quantum.Dagger(basis) * basis).astype(np.cdouble))
        ax.imshow(basis_test, vmin=0, cmap = 'hot', interpolation=None)
        ax.set_title('l=%d' % l)
    fig.suptitle(r'$%s$' % group_label)
    plt.tight_layout()
    plt.show()
    break

# `qet_to_irrep_basis` 

In [ ]:
qet = Qet({(2,-1):-1/sp.S(2), (2,0):1/sp.S(2), (2,1):1/sp.S(2)})
lmqet_to_irrepqet('O', qet)

In [ ]:
def lmqet_to_irrepqet_v0(group_label, qet, returnbasis=False):
    '''
    Given  a qet whose keys are (l,m) values, for a shared value
    of l this function determines the representation in terms of
    irreducible basis functions.
    
    Note    that    the    basis   obtained   from   irreducible
    representations   may  not  necessarily  be  orthonormal  or
    complete.
    
    Parameters
    ----------
    
    group_label : str
                A  label for crystallographic point group, must be
                one of CGPs.all_group_labels
    qet         : Qet
                A  Qet  whose  keys are assumed to be tuples (l,m)
                and whose values are complex numbers.
    returnbasis : bool
                If  True, the function also returns the list whose
                elements are the coefficients of the basis vectors
                used  in  the  decomposition.  This may be used to
                check if the basis is orthonormal.
    
    Returns
    -------

    tuple : (l, p_qet)
    
        l       : int
                The value of l for the input Qet.
        p_qet   : Qet
                A  Qet  whose  keys  are  labels  to components of
                irreducible   reps   and   whose  values  are  the
                corresponding coefficients. If the component label
                is  accompanied  by  a  left-superindex  then that
                number  in  parenthesis  is  indexing which of the
                basis this component is referring to.

    See Also
    --------
    irrepqet_to_lmqet

    Example
    -------
    
    >>> qet = Qet({(2,-1):-1/2, (2,0):1/2, (2,1):1/2})
    >>> print(lmqet_to_irrepqet('O', qet))
        (2,
        Qet({u_{E}  : 1/2, 
             y_{T_2}: -sqrt(2)/2}
           )
        )
    '''
    group = CPGs.get_group_by_label(group_label)
    irrep_basis = group.symmetry_adapted_bases
    irrep_labels = group.irrep_labels
    # for brevity I will only consider the case where there is a single l represented
    # in the qet
    ls = list(set((map(lambda x: x[0],qet.dict.keys()))))
    assert len(ls) == 1, "qet must be a supersposition of equal values of l"
    l = ls[0]
    l_basis = [(l,m) for m in range(-l,l+1)]
    # pick out all the parts for this value of l
    this_irrep_basis = {ir: irrep_basis[ir][l] for ir in irrep_labels}
    # kick out the ones that are empty
    this_irrep_basis = {k:v for k,v in this_irrep_basis.items() if len(v) != 0}
    size_of_basis = {k:len(v) for k, v in this_irrep_basis.items()}
    # convert the qets in the bases to coordinate vectors in the standard basis
    this_irrep_basis = {(irrep_label,basis_idx):list(map(lambda x: x.vec_in_basis(l_basis), basis)) \
                   for irrep_label, bases in this_irrep_basis.items() \
                   for basis_idx, basis in enumerate(bases)}
    # flatten and grab component labels
    nice_basis = {}
    
    for k, basis in this_irrep_basis.items():
        irrep_label = k[0]
        basis_idx = k[1]
        irrep_dim = group.irrep_dims[irrep_label]
        num_basis_vecs =  size_of_basis[irrep_label]
        if num_basis_vecs == 1:
            basis_labels = [(irrep_label,i,basis_idx,(group.component_labels[irrep_label][i])) for i in range((irrep_dim))]
        else:
            basis_labels = [(irrep_label,i,basis_idx,sp.Symbol(("{}^{(%d)}%s") % (basis_idx+1,str(group.component_labels[irrep_label][i])))) for i in range((irrep_dim))]
        nice_basis.update(OrderedDict(zip(basis_labels, basis)))
    # finally, proyect the qet onto the basis
    proyected_qet = {}
    c_qet = qet.vec_in_basis(l_basis)
    p_qet = {k:sp.Matrix(c_qet).dot(sp.Matrix(v)) for k,v in nice_basis.items()}
    # clean out bits that are zero, and convert to Qet
    p_qet = Qet({k[-1]: v for k,v in p_qet.items() if v!=0})
    if returnbasis:
        return l, p_qet, list(nice_basis.values())
    else:
        return l, p_qet

In [ ]:
lmax = 8
change_of_basis_matrices = {}
for group_label in CPGs.all_group_labels:
    print('\n%s\n%s\n%s' % ("#"*len(group_label),group_label,"#"*len(group_label)))
    group = CPGs.get_group_by_label(group_label)
    irrep_basis = group.symmetry_adapted_bases
    irrep_labels = group.irrep_labels
    all_matrices = {}
    bases_with_labels = {}
    for l in range(lmax+1):
        print('%s%d' % ("l = ",l),end=' | ')
        l_basis = [(l,m) for m in range(-l,l+1)]
        # pick out all the parts for this value of l
        this_irrep_basis = {ir: irrep_basis[ir][l] for ir in irrep_labels}
        # kick out the ones that are empty
        this_irrep_basis = {k:v for k,v in this_irrep_basis.items() if len(v) != 0}
        size_of_basis = {k:len(v) for k, v in this_irrep_basis.items()}
        # convert the qets in the bases to coordinate vectors in the standard basis
        this_irrep_basis = {(irrep_label,basis_idx):list(map(lambda x: x.vec_in_basis(l_basis), basis)) \
                       for irrep_label, bases in this_irrep_basis.items() \
                       for basis_idx, basis in enumerate(bases)}
        # flatten and grab component labels
        all_basis_labels = []
        full_basis_cols = []
        for k, basis in this_irrep_basis.items():
            irrep_label = k[0]
            basis_idx = k[1]
            irrep_dim = group.irrep_dims[irrep_label]
            num_basis_vecs =  size_of_basis[irrep_label]
            if num_basis_vecs == 1:
                basis_labels = [{'irrep_idx': i,
                                 'irrep_label': irrep_label,
                                 'basis_idx': basis_idx,
                                 'component_label':(group.component_labels[irrep_label][i])} for i in range((irrep_dim))]
            else:
                basis_labels = [{'irrep_idx': i,
                                 'irrep_label':irrep_label,
                                 'basis_idx': basis_idx,
                                 'component_label':sp.Symbol(("{}^{(%d)}%s") % \
                                                             (basis_idx+1,str(group.component_labels[irrep_label][i])))
                                } for i in range((irrep_dim))]
            all_basis_labels.extend(basis_labels)
            full_basis_cols.extend(basis)
        thedirectmatrix = sp.Matrix(full_basis_cols).T
        theinversematrix = sp.simplify(thedirectmatrix.inv())
        bases_with_labels[l] = {'basis_vec_symbols': all_basis_labels, 
                                'coord_transform_matrix_from_ir_to_lm': thedirectmatrix,
                                'coord_transform_matrix_from_lm_to_ir': theinversematrix}
        assert thedirectmatrix.rows == 2*l+1, "This ain't right."
    change_of_basis_matrices[group_label] = bases_with_labels


In [ ]:
qet = Qet({(2,-1):-sp.S(1)/2, (2,0):sp.S(1)/2, (2,1):sp.S(1)/2})
group_label = 'O'
group = CPGs.get_group_by_label(group_label)
# for brevity I will only consider the case where there is a single l represented
# in the qet
ls = list(set((map(lambda x: x[0],qet.dict.keys()))))
assert len(ls) == 1, "qet must be a supersposition of equal values of l"
l = ls[0]
l_basis = [(l,m) for m in range(-l,l+1)]
c_qet = sp.Matrix(qet.vec_in_basis(l_basis))
component_labels = [x['component_label'] for x in change_of_basis_matrices[group_label][l]['basis_vec_symbols']]
CB_matrix = change_of_basis_matrices[group_label][l]['coord_transform_matrix_from_lm_to_ir']
ir_coeffs = list(CB_matrix*c_qet)
qet_dict = {k:v for k,v in zip(component_labels, ir_coeffs) if v !=0}
ir_qet = Qet(qet_dict)
CB_matrix = change_of_basis_matrices[group_label][l]['coord_transform_matrix_from_ir_to_lm']
lm_coeffs = list(CB_matrix*sp.Matrix(ir_coeffs))
lm_dict = {k:v for k,v in zip(l_basis, lm_coeffs) if v !=0}
lm_qet = Qet(lm_dict)

In [ ]:
ir_qet.as_ket()

In [ ]:
ir_qet

In [ ]:
c_qet

In [ ]:
def lmqet_to_irrepqet_v1(group_label, qet):
    '''
    Given  a qet whose keys are (l,m) values, for a shared value
    of l this function determines the representation in terms of
    irreducible basis functions.
    
    Parameters
    ----------
    
    group_label : str
                A  label for crystallographic point group, must be
                one of CGPs.all_group_labels
    qet         : Qet
                A  Qet  whose  keys are assumed to be tuples (l,m)
                and whose values are complex numbers.
    
    Returns
    -------

    tuple : (l, p_qet)
    
        l       : int
                The value of l for the input Qet.
        p_qet   : Qet
                A  Qet  whose  keys  are  labels  to components of
                irreducible   reps   and   whose  values  are  the
                corresponding coefficients. If the component label
                is  accompanied  by  a  left-superindex  then that
                number  in  parenthesis  is  indexing which of the
                basis this component is referring to.

    See Also
    --------
    irrepqet_to_lmqet

    Example
    -------
    
    >>> qet = Qet({(2,-1):-1/2, (2,0):1/2, (2,1):1/2})
    >>> print(lmqet_to_irrepqet('O', qet))
        (2,
        Qet({u_{E}  : 1/2, 
             y_{T_2}: -sqrt(2)/2}
           )
        )
    '''
    group = CPGs.get_group_by_label(group_label)
    # for brevity I will only consider the case where there is a single l represented
    # in the qet
    ls = list(set((map(lambda x: x[0],qet.dict.keys()))))
    assert len(ls) == 1, "qet must be a supersposition of equal values of l"
    l = ls[0]
    l_basis = [(l,m) for m in range(-l,l+1)]
    c_qet = sp.Matrix(qet.vec_in_basis(l_basis))
    # pick out all the parts for this value of l
    this_irrep_basis = {ir: irrep_basis[ir][l] for ir in irrep_labels}
    # kick out the ones that are empty
    this_irrep_basis = {k:v for k,v in this_irrep_basis.items() if len(v) != 0}
    size_of_basis = {k:len(v) for k, v in this_irrep_basis.items()}
    # convert the qets in the bases to coordinate vectors in the standard basis
    this_irrep_basis = {(irrep_label,basis_idx):list(map(lambda x: x.vec_in_basis(l_basis), basis)) \
                   for irrep_label, bases in this_irrep_basis.items() \
                   for basis_idx, basis in enumerate(bases)}
    # flatten and grab component labels
    nice_basis = {}
    
    for k, basis in this_irrep_basis.items():
        irrep_label = k[0]
        basis_idx = k[1]
        irrep_dim = group.irrep_dims[irrep_label]
        num_basis_vecs =  size_of_basis[irrep_label]
        if num_basis_vecs == 1:
            basis_labels = [(irrep_label,i,basis_idx,(group.component_labels[irrep_label][i])) for i in range((irrep_dim))]
        else:
            basis_labels = [(irrep_label,i,basis_idx,sp.Symbol(("{}^{(%d)}%s") % (basis_idx+1,str(group.component_labels[irrep_label][i])))) for i in range((irrep_dim))]
        nice_basis.update(OrderedDict(zip(basis_labels, basis)))
    # finally, proyect the qet onto the basis
    proyected_qet = {}
    c_qet = qet.vec_in_basis(l_basis)
    p_qet = {k:sp.Matrix(c_qet).dot(sp.Matrix(v)) for k,v in nice_basis.items()}
    # clean out bits that are zero, and convert to Qet
    p_qet = Qet({k[-1]: v for k,v in p_qet.items() if v!=0})
    if returnbasis:
        return l, p_qet, list(nice_basis.values())
    else:
        return l, p_qet

In [ ]:
_,_,basis = (lmqet_to_irrepqet('O',lmqet,True))

In [ ]:
# check to see if the rank of the matrices matches
# the number of basis vectors this makes them be
# linearly independent
matrix_bases = {}
lmax = 8
for group_label in CPGs.all_group_labels:
    print(group_label, end=' || ')
    for l in range(lmax+1):
        dummy_ket = Qet({(l,0):1})
        _,_,basis = (lmqet_to_irrepqet(group_label,dummy_ket,True))
        check = (sp.Matrix(basis).rank()==len(basis))
        matrix_bases[(group_label, l)] = (l, basis, len(basis), check)

In [ ]:
checks = [k[-1] for k in matrix_bases.values()]
print('rank checks : %d/%d' %(sum(checks),len(checks)))
dim_checks = [(k[2] == (2*k[0]+1)) for k in matrix_bases.values()]
print('dim  checks : %d/%d' %(sum(dim_checks),len(dim_checks)))

In [ ]:
from itertools import combinations

In [ ]:
lmax = 8
for group_label in CPGs.all_group_labels:
    fig, axes = plt.subplots(nrows=2, ncols=4,figsize=(9,6))
    for l in range(1,lmax+1):
        ax = axes[(l-1)//4,(l-1)%4]
        basis = matrix_bases[(group_label, l)][1]
        basis_test = np.abs(np.array(sp.Matrix(basis) * sp.physics.quantum.Dagger(sp.Matrix(basis))).astype(np.cdouble))
        ax.imshow(basis_test, vmin=0, vmax=1, cmap = 'hot', interpolation=None)
        ax.set_title('l=%d' % l)
    fig.suptitle(r'$%s$' % group_label)
    plt.tight_layout()
    plt.show()

In [ ]:
def cosY(l,m,coeff):
    if m == 0:
        return sp.S(coeff)*Qet({(l,0):1})
    return sp.S(coeff)/sp.sqrt(2)*Qet({(l,m):1,(l,-m):1})
def sinY(l,m,coeff):
    if m == 0:
        return sp.S(coeff)*Qet({(l,0):1})
    return sp.I*sp.S(coeff)/sp.sqrt(2)*Qet({(l,m):1,(l,-m):-1})

In [ ]:
# bradley basis for l=5
bs = {}
l = 5
bs[sp.Symbol('A_1')] = []
bs[sp.Symbol('A_2')] = []
bs[sp.Symbol('E')] = [[sinY(5,4,1),sinY(5,2,-1)]]
bs[sp.Symbol('T_1')] = [[cosY(l,1,0.484)+cosY(l,3,-0.523)+cosY(l,5,0.702),
                         sinY(l,1,0.484)+sinY(l,3,0.523)+sinY(l,5,0.702),
                         cosY(l,0,1)],
                        [cosY(l,1,0.573)+cosY(l,3,0.795)+cosY(l,5,0.197),
                         sinY(l,1,0.573)+sinY(l,3,-0.795)+sinY(l,5,0.197),
                         cosY(l,4,1)]] 
bs[sp.Symbol('T_2')] = [[cosY(l,1,0.661)+cosY(l,3,-0.306)+cosY(l,5,-0.684),
                         sinY(l,1,-0.661)+sinY(l,3,-0.306)+sinY(l,5,0.684),
                         cosY(l,2,1)]]
basis_5 = [(l,m) for m in range(-l,l+1)]
all_components = []
for irrep_symbol in bs.keys():
    for component in bs[irrep_symbol]:
        all_components.extend([c.vec_in_basis(basis_5) for c in component])
tee = sp.Matrix(all_components)

In [ ]:
basis_5_mine = {k: [v1 for k1,v1 in v.items() if k1 == 5] for k,v in symmetry_bases['O'].items()}

In [ ]:
all_my_components = []
for irrep_symbol in basis_5_mine.keys():
    for component in basis_5_mine[irrep_symbol]:
        for c in component:
            all_my_components.extend([zee.vec_in_basis(basis_5) for zee in c])
all_my_components = sp.Matrix(all_my_components)

In [ ]:

# fig, ax = plt.subplots(figsize=(4,4))
# basis = matrix_bases[(group_label, l)][1]
# ax.imshow(np.abs(tee).astype(np.float64), vmin=0, vmax=1, cmap = 'hot', interpolation=None)
# ax.set_title('l=%d' % l)
# fig.suptitle(r'$%s$' % group_label)
# plt.tight_layout()
# plt.show()

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(8,4))
axes[0].imshow(np.abs(all_my_components*sp.physics.quantum.Dagger(all_my_components)).astype(np.float64), vmin=0, vmax=1, cmap = 'hot', interpolation=None)
axes[0].set_title('my basis')
axes[1].imshow(np.abs(np.array(tee*sp.physics.quantum.Dagger(tee))).astype(np.float64)
           ,vmin=0, vmax=1, cmap = 'hot', interpolation=None)
axes[1].set_title('B&C basis')
# plt.tight_layout()
plt.show()

In [ ]:
group_label = 'O'
lmqet = Qet({(2,-2):sp.Symbol('a'), (2,0): sp.S(1)/sp.sqrt(3), (2,2): sp.S(1)/sp.sqrt(3)})
display(lmqet.as_ket())
l, irrep_qet = lmqet_to_irrepqet(group_label,lmqet)
display(irrep_qet.as_ket())
sp.simplify(irrepqet_to_lmqet(group_label, irrep_qet, 2)[1].as_ket())

In [ ]:
def irrepqet_to_lmqet(group_label, irrep_qet, l, returnbasis=False):
    '''
    This  function  take  a  label  for a crystallographic point
    group  and  a  qet  written in an irreducible representation
    basis,  understood  to  originate from the given value of l,
    and  it  returns  the  representation  in the standard (l,m)
    basis.
    
    Parameters
    ----------
    
    group_label : str
                A  label for crystallographic point group, must be
                one of CGPs.all_group_labels
    irrep_qet   : Qet
                A  Qet  whose  keys  are  assumed labels for irrep
                components    and   whose   values   are   complex
                coefficients.
    returnbasis : bool
                For debugging purposes only. If True, the function
                also  returns  the  list  whose  elements  are the
                coefficients  of  the  basis  vectors  used in the
                decomposition.
    
    Returns
    -------
    tuple : (l, p_qet)
    
        l       : int
                The value of l for the input Qet.
       lm_qet   : Qet
                A  Qet  whose  keys  are  (l,m) tuples and whose
                values are complex coefficients.
                
    See Also
    --------
    lmqet_to_irrepqet
    
    Example
    -------
    
    >>> irrep_qet = Qet({sp.Symbol('u_{E}'):1/2, sp.Symbol('y_{T_2}'):-sp.sqrt(2)/2})
    >>> print(irrepqet_to_lmqet('O', irrep_qet, 2))
        (2,
        Qet({(2,-1):-1/2,
             (2,0) :1/2,
             (2,1) :1/2})
        )
    '''
    group = CPGs.get_group_by_label(group_label)
    irrep_basis = group.symmetry_adapted_bases
    irrep_labels = group.irrep_labels
    l_basis = [(l,m) for m in range(-l,l+1)]
    # pick out all the parts for this value of l
    this_irrep_basis = {ir: irrep_basis[ir][l] for ir in irrep_labels}
    # kick out the ones that are empty
    this_irrep_basis = {k:v for k,v in this_irrep_basis.items() if len(v) != 0}
    size_of_basis = {k:len(v) for k, v in this_irrep_basis.items()}
    # convert the qets in the bases to coordinate vectors in the standard basis
    this_irrep_basis = {(irrep_label,basis_idx):list(map(lambda x: x.vec_in_basis(l_basis), basis)) \
                   for irrep_label, bases in this_irrep_basis.items() \
                   for basis_idx, basis in enumerate(bases)}
    # flatten and grab component labels
    nice_basis = {}
    
    for k, basis in this_irrep_basis.items():
        irrep_label = k[0]
        basis_idx = k[1]
        irrep_dim = group.irrep_dims[irrep_label]
        num_basis_vecs =  size_of_basis[irrep_label]
        if num_basis_vecs == 1:
            basis_labels = [((group.component_labels[irrep_label][i])) for i in range((irrep_dim))]
        else:
            basis_labels = [(sp.Symbol(("{}^{(%d)}%s") % (basis_idx+1,str(group.component_labels[irrep_label][i])))) for i in range((irrep_dim))]
        nice_basis.update(OrderedDict(zip(basis_labels, basis)))
    lm_qet = [sp.Matrix(nice_basis[k])*v for k,v in irrep_qet.dict.items()]
    lm_qet = sum(lm_qet, sp.Matrix([0]*(2*l+1)))
    lm_qet = Qet(dict(zip(l_basis,lm_qet)))
    if returnbasis:
        return l, lm_qet, list(nice_basis.values())
    else:
        return l, lm_qet


# Change pos-facto approach

It truly seems to me that the simplest way of implementing this is via a coordinate transformation at the end of the previous calculation, not to start everything from scratch.

In [ ]:
l = 2
this_basis = symmetry_bases['C_{1}'][sp.Symbol('A')][l]
full_basis = [(l,m) for m in range(-l,l+1)]
# find coordinates in standard basis
coord_vec = sp.Matrix(this_basis[2][0].vec_in_basis(full_basis))
# compute the change of coordinate matrix
combos = [Qet({
    (l,m):sp.S(1)/sp.sqrt(2),
    (l,-m):sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
combos.append(Qet({(l,0):1}).vec_in_basis(full_basis))
combos = combos + [Qet({
    (l,m):sp.I*sp.S(1)/sp.sqrt(2),
    (l,-m):-sp.I*sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
change_of_basis_matrix = sp.Matrix(combos)
full_qet_basis = [Qet({(l,m):1}) for m in range(-l,l+1)]
transformed_vec = list(change_of_basis_matrix*coord_vec)
transformed_qet = sum([v*q for v,q in zip(transformed_vec, full_qet_basis) if v !=0],Qet({}))


In [ ]:
this_basis = symmetry_bases['C_{1}']
new_basis = {}
for irrep, ldict in this_basis.items():
    new_basis[irrep] = {}
    for l, qet_groups in ldict.items():
        new_basis[irrep][l] = []
        full_basis = [(l,m) for m in range(-l,l+1)]
        combos = [Qet({
            (l,m):sp.S(1)/sp.sqrt(2),
            (l,-m):sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
        combos.append(Qet({(l,0):1}).vec_in_basis(full_basis))
        combos = combos + [Qet({
            (l,m):sp.I*sp.S(1)/sp.sqrt(2),
            (l,-m):-sp.I*sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
        change_of_basis_matrix = sp.Matrix(combos)
        full_qet_basis = [Qet({(l,m):1}) for m in range(-l,l+1)]
        for qet_group in qet_groups:
            new_qets = []
            for qet in qet_group:
                coord_vec = sp.Matrix(qet.vec_in_basis(full_basis))
                transformed_vec = list(change_of_basis_matrix*coord_vec)
                transformed_qet = sum([v*q for v,q in zip(transformed_vec, full_qet_basis) if v !=0],Qet({}))
                new_qets.append(transformed_qet)
            new_basis[irrep][l].append(new_qets)


In [ ]:
all_new_bases = {}
for group_label, this_basis in symmetry_bases.items():
    new_basis = {}
    for irrep, ldict in this_basis.items():
        new_basis[irrep] = {}
        for l, qet_groups in ldict.items():
            new_basis[irrep][l] = []
            full_basis = [(l,m) for m in range(-l,l+1)]
            combos = [Qet({
                (l,m):sp.S(1)/sp.sqrt(2),
                (l,-m):sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
            combos.append(Qet({(l,0):1}).vec_in_basis(full_basis))
            combos = combos + [Qet({
                (l,m):sp.I*sp.S(1)/sp.sqrt(2),
                (l,-m):-sp.I*sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
            change_of_basis_matrix = sp.Matrix(combos)
            full_qet_basis = [Qet({(l,m):1}) for m in range(-l,l+1)]
            for qet_group in qet_groups:
                new_qets = []
                for qet in qet_group:
                    coord_vec = sp.Matrix(qet.vec_in_basis(full_basis))
                    transformed_vec = list(change_of_basis_matrix*coord_vec)
                    transformed_qet = sum([v*q for v,q in zip(transformed_vec, full_qet_basis) if v !=0],Qet({}))
                    new_qets.append(transformed_qet)
                new_basis[irrep][l].append(new_qets)
    all_new_bases[group_label] = new_basis


In [ ]:
# ugh this simplifies some things, but complexifies others ...

In [ ]:
all_new_bases['C_{1}'][sp.Symbol('A')]

In [ ]:
symmetry_bases['O'][sp.Symbol('T_2')][2]

In [ ]:
all_new_bases['O'][sp.Symbol('T_2')][2]

In [ ]:
l = 2
this_basis = symmetry_bases['C_{1}'][sp.Symbol('A')][l]
full_basis = [(l,m) for m in range(-l,l+1)]
# find coordinates in standard basis
coord_vec = sp.Matrix(this_basis[0][0].vec_in_basis(full_basis))
# compute the change of coordinate matrix
combos = [Qet({
    (l,m):sp.S(1)/sp.sqrt(2),
    (l,-m):sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
combos.append(Qet({(l,0):1}).vec_in_basis(full_basis))
combos = combos + [Qet({
    (l,m):sp.I*sp.S(1)/sp.sqrt(2),
    (l,-m):-sp.I*sp.S(1)/sp.sqrt(2)}).vec_in_basis(full_basis) for m in range(1,l+1)]
change_of_basis_matrix = sp.Matrix(combos)
full_qet_basis = [Qet({(l,m):1}) for m in range(-l,l+1)]
transformed_vec = list(change_of_basis_matrix*coord_vec)
transformed_qet = sum([v*q for v,q in zip(transformed_vec, full_qet_basis) if v !=0],Qet({}))


In [ ]:
from sympy.physics.quantum.dagger import Dagger

In [ ]:
change_of_basis_matrix*Dagger(change_of_basis_matrix)

In [ ]:
cosine_combos

# Visualizing Surface Harmonics

Much better to plot in Mathematica, must create something that is easily importable there.

In [ ]:
import multiprocessing
import time
from joblib import Parallel, delayed
num_cores = multiprocessing.cpu_count()

In [ ]:
lmax = 12
irreps_ls = Parallel(n_jobs = num_cores)(delayed(irrep_basis_from_Ylm)('O', l) for l in range(lmax+1))

In [ ]:
stringos = []
group_label = 'O'
group = CPGs.get_group_by_label(group_label)
irrep_labels = group.irrep_labels
ls = []
lmax = 12
for l in range(lmax+1):
    qets = irreps_ls[l]
    try:
        math_export= '},{'.join([','.join([str(k[0]),str(k[1]),str(v).replace('sqrt','Sqrt').replace('(','[').replace(')',']')]) for k,v in qets[irrep_labels[0]][0].dict.items()])
        math_export = 'l%d={{%s}};'%(l,math_export)
        stringos.append(math_export)
        ls.append(l)
    except:
        pass
coda = 'ls={%s};' % (','.join(list(map(lambda x: 'l%d' % x,ls))))
stringos.append(coda)
pypercopy('\n'.join(stringos))

In [ ]:
from wolframclient.evaluation import WolframLanguageSession
import io
from wolframclient.language import wlexpr
from PIL import Image
class Mathematica():
    def __init__(self):
        self.session = WolframLanguageSession()
    def plot(self, plot_code):
        _ = self.evaluate(plot_code)
        data = self.evaluate( wlexpr('ExportByteArray[ plot, "PNG" ]'))
        return Image.open(io.BytesIO(data))
    def evaluate(self, code):
        return self.session.evaluate(code)

In [ ]:
mathematica = Mathematica()

In [ ]:
chunk_of_code = '\n'.join(stringos+['''
Os = Sum[
    p[[3]]*SphericalHarmonicY[p[[1]], 
    p[[2]], \[Theta], \[Phi]], {p, #}] & /@ ls;
sym = ParallelTable[
   Rasterize[
    SphericalPlot3D[
     Re[Os[[2]]], {\[Theta], 0, \[Pi]}, {\[Phi], 0, 2 \[Pi]}, 
     PlotRange -> All, PlotPoints -> 30, SphericalRegion -> True, 
     Boxed -> False, Axes -> False, 
     PlotStyle -> MaterialShading["Gold"], 
     ViewVector -> {5 Cos[t], 5 Sin[t], 10 Sin[t/2]}
                   ]
     ],
    {t, 0, 4 \[Pi], 2 \[Pi]/18}
   ];
plot = GraphicsGrid[Partition[sym, 6], ImageSize -> 800];
'''])

In [ ]:
mathematica.plot(chunk_of_code)